In [1]:
from bs4 import BeautifulSoup
import requests as rq
import re
import datetime
import pymysql
from flask import Flask,jsonify, request
# from fake_useragent import UserAgent
import cv2
import numpy as np
import time
from urllib.parse import urlencode
import threading
import sys
import os
import json
import shutil
import traceback
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import base64

In [2]:
# db_settings = {
#     "host": "127.0.0.1",
#     "port": 3306,
#     "user": "root",
#     "password": "As123459362",
#     "db": "pttcrawler",
#     "charset": "utf8",
#     "autocommit":True
# }
db_settings = {
    "host": "60.250.109.71",
    "port": 23306,
    "user": "xuan",
    "password": "Qaz123",
    "db": "tender",
    "charset": "utf8mb4",
    "autocommit":True
}
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
    "Accept-Encoding": "gzip, deflate, br", 
    "Accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6", 
    "Host": "web.pcc.gov.tw",  #目標網站 
    "Connection":"close",
    "Sec-Fetch-Dest": "document", 
    "Sec-Fetch-Mode": "navigate", 
    "Sec-Fetch-Site": "same-origin", 
    "Upgrade-Insecure-Requests": "?1", 
    "User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'
     #使用者代理
}

In [3]:
#lxml pyinstaller編不動，nuitka不行
#parser = "lxml"
parser = "html.parser"
parser_detail = 'html5lib'

In [4]:
#創建一個資料夾存放驗證用撲克牌圖片
path = './temp_validate_img'
if os.path.exists(path):
    shutil.rmtree (path)
    os.mkdir(path)
else:
    os.mkdir(path)

In [5]:
# url = 'https://web.pcc.gov.tw/tps/tpam/main/tps/tpam/tpam_tender_detail.do?searchMode=common&scope=F&primaryKey=53759591'
# urll='https://icanhazip.com/'
# s = rq.session()
# r = s.get(urll, headers = headers,verify=False,proxies={'https':'http://170.155.5.235:8080'}, timeout=(4,7))

In [6]:
# 從免費代理網站取得代理ip
def get_proxy_ip(isHttps):
    url = {"usproxy":"https://www.us-proxy.org/","freeproxylist_sslproxy":"https://www.sslproxies.org/"}
    metas=[]
    for i in url:
        print(i)
        r = rq.get(url[i],verify=False)
        html_doc = r.text
        soup = BeautifulSoup(html_doc, parser)
        trs = soup.select("table.table.table-striped.table-bordered tr")  
        #print(trs)
        for tr in trs:
            tds = tr.select("td")
            #print(tds)
            #print(len(tds))
            if len(tds) > 6:
                ifScheme = tds[6].text
                ip = tds[0].text
                port = tds[1].text
                anonymity = tds[4].text
                proxy = "%s:%s"%(ip, port)
                meta = {
                    'proxyIp': proxy,
                    'connect_times':1,
                    'successful_connect_times':0,
                    'qual_ratio':0,
                    'isDelete':False,
                    'fromm':i,
                    'mine' : None
                  }
                if ifScheme == isHttps:
                    if ifScheme == "yes":
                        meta['ishttps'] = True
                        # https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#https-proxy-error-http-proxy
                        meta['proxyIp'] = "http://" + meta['proxyIp']
                    elif ifScheme == "no":
                        meta['ishttps'] = False
                        meta['proxyIp'] = "http://" + meta['proxyIp']
                    meta = test_proxy(meta,timeout_sec=3)
                    metas.append(meta)
                    continue                    
    #------------------------------------
    headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
    "Accept-Encoding": "gzip, deflate, br", 
    "Accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6", 
    "Host": "free-proxy.cz",  #目標網站 
    "Sec-Fetch-Dest": "document", 
    "Sec-Fetch-Mode": "navigate", 
    "Sec-Fetch-Site": "same-origin", 
    "Upgrade-Insecure-Requests": "?1", 
    "User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'
     #使用者代理
}
    
    if isHttps =="yes":
        pp = 'https'
    elif isHttps =="no":
        pp = 'http'
    with rq.session() as s:
        s.keep_alive = True
        for j in range(1,4):
            try:
                r = s.get(f"http://free-proxy.cz/zh/",headers=headers,timeout=10)

                headers['Content-Type']='application/x-www-form-urlencoded'
                data=f"country=all&protocol={pp}&anonymity=all&send=Filter+proxies"
                r = s.post("http://free-proxy.cz/en/?do=searchFilter-submit",data=data,headers=headers,timeout=10)
                #http://free-proxy.cz/en/proxylist/country/all/https/ping/all/2
                if j >1:
                    headers['Referer']=f'http://free-proxy.cz/en/proxylist/country/all/{pp}/ping/all'
                    headers['Connection']=f'keep-alive'
                    r = s.get(f"http://free-proxy.cz/zh/proxylist/country/all/{pp}/ping/all/{j}",headers=headers,timeout=10,verify=False)
                print("free-proxy.cz抓取頁數： "+str(j))
            except Exception as e:
                print(e)
                print("free-proxy.cz 無法進入")
                continue
            print(r.url)
            html_doc = r.text
            soup = BeautifulSoup(html_doc, parser)
            proxylist = soup.select('table#proxy_list tr')[1:]
            print(len(proxylist))
            for i in proxylist:
                try:
                    try:
                        ip = base64.b64decode(re.search(r'Base64\.decode\(\"(.*)\"\)',str(i.select('td')[0])).group(1)).decode("utf-8")
                    except AttributeError :
                        continue
                    port = i.select('td')[1].text
                    Protocol  = i.select('td')[2].text.lower()
                    place=i.select('td')[2].text.lower()
                    meta = {
                        'proxyIp': 'http://'+ str(ip) +':'+ str(port),
                        'connect_times':1,
                        'successful_connect_times':0,
                        'qual_ratio':0,
                        'isDelete':False,
                        'fromm':'freeProxyCZ',
                        'mine' : None
                      }
                    if Protocol =='https' and isHttps =="yes":
                        meta['ishttps'] = True
                        # 測了就抓不到下一頁了
                        #meta = test_proxy(meta,timeout_sec=3)
                        meta['isValidate'] = False
                        metas.append(meta)
                    elif Protocol =='http' and isHttps =="no":
                        meta['ishttps'] = False
                        #meta = test_proxy(meta,timeout_sec=3)
                        meta['isValidate'] = False
                        metas.append(meta)
                except:
                    continue
                if len(proxylist)<37:
                    break
    return metas

In [7]:
# 測試 proxy 可用性
def test_proxy(proxy,timeout_sec,headers={}):
    #print(proxy['proxyIp'])
    with rq.session() as s:
        try:
            if proxy['ishttps'] == True:
                url = "https://icanhazip.com/"
                r = s.get(url, headers = headers,proxies={'https':proxy['proxyIp'],},verify=False, timeout=timeout_sec)
            elif proxy['ishttps'] == False:
                url = "http://icanhazip.com/"
                r = s.get(url, headers = headers,proxies={'http':proxy['proxyIp'],},verify=False, timeout=timeout_sec)
            print(r.status_code)
            print(proxy['proxyIp'])
            if r.status_code==200:
                proxy['isValidate']=True
                proxy['successful_connect_times']=1
                proxy['qual_ratio']=1
            else:
                proxy['isValidate']=False
        except Exception as ex:
            proxy['isValidate']=False
            #print(ex)
        #print("---------------")
    return proxy

In [8]:
# 將取得的代理ip寫入資料庫，此函數也可以更新已寫入資料庫的資料
def send_ip(proxies_pool_https, db_settings,typee="",send_log = True,new = False):
    time_start = datetime.datetime.now() #開始計時
    data_combine=[]
    wrong = 0
    try:
        # 建立Connection物件
        conn = pymysql.connect(**db_settings)
        # 建立Cursor物件
        with conn.cursor() as cursor:
          #資料表相關操作
            get_max_log_id="select MAX(UID) FROM log"
            # % 操作符只能直接用於字串(‘123’)，列表([1,2,3])、元組
            command = "INSERT INTO proxy_ip(log_UID, proxyIp, connect_times, successful_connect_times, qual_ratio, isValidate, ishttps,isDelete, fromm)VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s) on DUPLICATE KEY UPDATE log_UID = values(log_UID),connect_times =connect_times+values(connect_times),successful_connect_times=successful_connect_times+values(successful_connect_times),qual_ratio=((successful_connect_times+values(successful_connect_times))/(connect_times+values(connect_times))),isValidate = values(isValidate),ishttps=values(ishttps),isDelete=values(isDelete)"
            if new:
                command = "INSERT INTO proxy_ip(log_UID, proxyIp, connect_times, successful_connect_times, qual_ratio, isValidate, ishttps,isDelete, fromm)VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s) on DUPLICATE KEY UPDATE log_UID = values(log_UID),connect_times = values(connect_times),successful_connect_times=values(successful_connect_times),qual_ratio=values(qual_ratio),isValidate = values(isValidate),ishttps=values(ishttps),isDelete=values(isDelete)"
            command_log = "INSERT INTO log(datetime, task, status, record_des, errmsg) VALUES(%s, %s, %s, %s, %s)"
            # 紀錄開始 (暫時取消)
#             if send_log:
#                 cursor.execute(command_log, (datetime.datetime.now(), "proxy_ip", "start", typee, ""))
            # 取得 log 的 UID
            cursor.execute(get_max_log_id)
            log_UID = str(cursor.fetchone()[0])
            # 組合數據
            if type(proxies_pool_https) == list:
                for proxy in proxies_pool_https:
                    data_combine.append((log_UID, proxy["proxyIp"], proxy["connect_times"], proxy["successful_connect_times"], proxy["qual_ratio"], proxy["isValidate"], proxy["ishttps"], proxy["isDelete"], proxy["fromm"]))  # 注意要用两个括号扩起来
            else:
                data_combine.append((log_UID, proxies_pool_https["proxyIp"], proxies_pool_https["connect_times"], proxies_pool_https["successful_connect_times"], proxies_pool_https["qual_ratio"], proxies_pool_https["isValidate"], proxies_pool_https["ishttps"], proxies_pool_https["isDelete"], proxies_pool_https["fromm"]))  # 注意要用两个括号扩起来
            # 執行
            try:
                cursor.executemany(command, data_combine)
            except Exception as err:
                print(err)
                wrong = wrong + 1
                print(wrong)
                cursor.execute(command_log, (datetime.datetime.now(), "proxy_ip", "wrong" + str(wrong), "", str(err)))
    except Exception as ex:
        print(ex)
        print(data_combine)
    time_end = datetime.datetime.now()    #結束計時
    time_c= time_end - time_start   #執行所花時間
    #print('proxy ip寫入資料庫，time cost', time_c, 's')

In [9]:
#p_yes=get_proxy_ip("yes")
#send_ip(p_yes, db_settings ,"only https",new = True)

In [10]:
# 從資料庫取出ip，未來有自架proxy的話，可以把proxy放到資料庫內，並設定 mine = 1，這樣就會固定抓這個proxy來用，如果沒有自架proxy的話，就還需要多寫 proxy expire 的處置方式。
def get_ip_from_db(isHttps,db_settings,qual_ratio=0,connect_times=4,new=True,mine_ip_use_interval=0):
    try:
        # 建立Connection物件
        data=[]
        conn = pymysql.connect(**db_settings)
        # 建立Cursor物件
        with conn.cursor(pymysql.cursors.DictCursor) as cursor:
          #資料表相關操作
            if isHttps=="yes":
                get_ip=f"select * FROM proxy_ip where isHttps = 1 and isDelete is False and ((qual_ratio > {qual_ratio} or connect_times < {connect_times}) or (mine = 1 and TIMESTAMPDIFF(second, updated_time, now())>{mine_ip_use_interval}))"
            elif isHttps=="no":
                get_ip=f"select * FROM proxy_ip where isHttps = 0 and isDelete is False and ((qual_ratio > {qual_ratio} or connect_times < {connect_times}) or (mine = 1 and TIMESTAMPDIFF(second, updated_time, now())>{mine_ip_use_interval}))"
            cursor.execute(get_ip)
            datas = cursor.fetchall()
        if new:
            for data in datas:
                data['successful_connect_times']=0
                data['connect_times']=0
                data['qual_ratio']=0
        return datas
    except Exception as ex:
        print(f'ip抓取失敗：{ex}')

In [11]:
#更新 mine proxy_ip 的可取用時間
def update_mine_proxy_ip(proxy_ip,db_settings):
    conn = pymysql.connect(**db_settings)
    with conn.cursor(pymysql.cursors.DictCursor) as cursor:
        update_proxy_ip = f"update proxy_ip SET updated_time=now() where proxyIp = '{proxy_ip['proxyIp']}'"
        cursor.execute(update_proxy_ip)

In [12]:
#撲克牌圖片驗證用算法
def pHash(img):
    # 感知哈希算法
    # 缩放32*32
    img = cv2.resize(img, (32, 32))   # , interpolation=cv2.INTER_CUBIC
 
    # 转换为灰度图
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # 将灰度图转为浮点型，再进行dct变换
    dct = cv2.dct(np.float32(gray))
    # opencv实现的掩码操作
    dct_roi = dct[0:8, 0:8]
 
    hash = []
    avreage = np.mean(dct_roi)
    for i in range(dct_roi.shape[0]):
        for j in range(dct_roi.shape[1]):
            if dct_roi[i, j] > avreage:
                hash.append(1)
            else:
                hash.append(0)
    return hash

In [13]:
#撲克牌圖片驗證用算法 - 比較
def cmpHash(hash1, hash2):
    # Hash值对比
    # 算法中1和0顺序组合起来的即是图片的指纹hash。顺序不固定，但是比较的时候必须是相同的顺序。
    # 对比两幅图的指纹，计算汉明距离，即两个64位的hash值有多少是不一样的，不同的位数越小，图片越相似
    # 汉明距离：一组二进制数据变成另一组数据所需要的步骤，可以衡量两图的差异，汉明距离越小，则相似度越高。汉明距离为0，即两张图片完全一样
    n = 0
    # hash长度不同则返回-1代表传参出错
    if len(hash1) != len(hash2):
        return -1
    # 遍历判断
    for i in range(len(hash1)):
        # 不相等则n计数+1，n最终为相似度
        if hash1[i] != hash2[i]:
            n = n + 1
    return n

In [14]:
#進行撲克牌驗證
def validate_ip(s,url,proxies):
    while True:
        response = s.get(url, proxies=proxies)
        html_doc = response.text
        soup = BeautifulSoup(html_doc, parser)
        img_url_d = soup.select('form#validateForm img')
        img_url_head = "http://web.pcc.gov.tw/tps/tpam/"
        img_data = []
        img_data.append({'img_src':img_url_head +img_url_d[0]['src'],'id':'question'})
        for j in img_url_d[2:]:
            d = {'img_src':img_url_head +j['src'],'id':j['alt']}
            img_data.append(d)
        right_hash_com =[]
        left_hash_com =[]
        
        path = './temp_validate_img'
        for k in img_data:
            response = s.get(k['img_src'])
            with open(path +'/'+ k['id']+'.jpg', 'wb') as file:
                file.write(response.content)
                file.close() 
            if k ==img_data[0]:
                img = cv2.imread(path +'/'+"question.jpg")
                crop_img = img[0:96, 6:77]
                ret,crop_img = cv2.threshold(crop_img,127,255,cv2.THRESH_BINARY)
                right_imageVar = pHash(crop_img)
                crop_img = img[0:96, 89:160]
                ret,crop_img = cv2.threshold(crop_img,127,255,cv2.THRESH_BINARY)
                left_imageVar = pHash(crop_img)
            image = cv2.imread(path +'/'+k['id']+'.jpg')
            ret,image = cv2.threshold(image,127,255,cv2.THRESH_BINARY)
            right = cmpHash(pHash(image),right_imageVar)
            left = cmpHash(pHash(image),left_imageVar)
            right_hash_com.append(right)
            left_hash_com.append(left)

        right_idx = np.argmin(right_hash_com[1:])
        left_idx = np.argmin(left_hash_com[1:])
        data="choose=" +img_data[right_idx+1]['id']+"&choose="+img_data[left_idx+1]['id']+"&id="+re.match(r'.*id=(.*)', url, flags=0).group(1)
        response = s.post(url,headers = {'Content-Type':'application/x-www-form-urlencoded'}, proxies=proxies,data=data, timeout=10)
        if re.search("為預防惡意程式針對本系統進行大量查詢致影響系統服務品質",response.text):
            print(f"再次檢核：{response.url}")
            url=response.url
        elif not re.search("為預防惡意程式針對本系統進行大量查詢致影響系統服務品質",response.text) and response.status_code == 200:
            print("通過驗證")
            #print(response.text)
            break
        else:
            #print(response.text)
            print(f"驗證失敗，{response.status_code}")

    return response

In [15]:
#取出字典某 key 值對應 value，若找無此 key 值，則回傳空字串 
def dicMemberCheck(key, dicObj):
    if key in dicObj:
        return dicObj[key]
    else:
        return ""
# dicMemberCheck('標案案號', {'標案案號':123,'ji3':456})

In [16]:
# href = "https://web.pcc.gov.tw/tps/sap/pcqm/main/sap/pcqm/showQuot.do?method=showDetail&searchMode=common&searchType=basic&pkPctmRfqMaster=50004351"
# s = rq.session()
# proxies={}
# refresh_proxies={}
# ran=None
# (xx,yy,cc,zz,ff) = get_tenderDeclaration_detail(s,href,ff,proxies,refresh_proxies,db_settings,diff_seconds_conn=0,headers={},typee="",No_proxy=True)

In [17]:
# tenderType = 'final'
# conn = pymysql.connect(**db_settings)
# get_data="SELECT * FROM tender.gov_purchase where (detail_data_html ='' and detail_data like '%{}%') and tenderType='"+tenderType+"'"
# # 建立Cursor物件
# with conn.cursor(pymysql.cursors.DictCursor) as cursor:
#   #資料表相關操作
#     cursor.execute(get_data)
#     raw_datas_all = cursor.fetchall()
# raw_datas = raw_datas_all[0:10]
#get_only_detail(tenderType,raw_datas,0,0,headers, db_settings,No_proxy=False)

In [18]:
# 抓招標、決標內頁資料
def get_tenderDeclaration_detail(s,href,ran,proxies,refresh_proxies,db_settings,diff_seconds_conn=1.2,headers={},typee="",No_proxy=False):
    detail_data_html=""
    detail_data={}
    global detail_waitt
    detail_waitt = 0
    #with requests.Session() as s:
    headers["User-Agent"] = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'
    breakdown = "no"
    time.sleep(diff_seconds_conn)
    if No_proxy:
        while detail_waitt:
            localtime = time.localtime()
            result = time.strftime("%Y%m%d_%H%M%S", localtime)
            print(typee + f"暫停休息 {60*detail_waitt}秒 一下，有執行緒被鎖，目前時間：{result}")
            time.sleep(60*detail_waitt)
        while True:
            try:
                r = s.get(href, headers = headers,verify=False, timeout=(4,7))
                if re.match(r'.*validate.*',r.url, flags=0):
                    print(f"本機 IP 準備執行驗證")
                    r = validate_ip(s,r.url,proxies={})
                if r.status_code<300 and (re.search(r'公告日',r.text) or re.search(r'標案案號',r.text)):
                    detail_waitt = 0
                    break
                elif re.search(r'系統發生錯誤',r.text):
                    detail_waitt = 0
                    detail_data="系統發生錯誤"
                    breakdown = "yes"
                    break
            except rq.exceptions.RequestException as ex:
                print("本機 ip 遭到封鎖")
                detail_waitt = detail_waitt + 1
                localtime = time.localtime()
                result = time.strftime("%Y%m%d_%H%M%S", localtime)
                print(typee + f"暫停休息 {800*detail_waitt}秒 一下，本機 IP 被鎖，目前時間：{result}")
                print(ex)
                time.sleep(800*detail_waitt)
            except Exception as ex:
                print(f'其他錯誤，{ex}')
                break
    else:
        while True:
            if not ran:
                ran = np.random.randint(len(proxies))
            try:
                proxies[ran]['connect_times'] =  proxies[ran]['connect_times'] + 1
                proxies[ran]['qual_ratio'] = proxies[ran]['successful_connect_times'] / proxies[ran]['connect_times']
                #print(f"{proxies[ran]['proxyIp']} 準備進入，{href}")                 
                r = s.get(href, headers = headers,verify=False,proxies={'https':proxies[ran]['proxyIp']}, timeout=(4,7))
                #print("內頁取得點存活")
                if re.match(r'.*validate.*',r.url, flags=0):
                    print(f"{proxies[ran]['proxyIp']} 準備執行驗證")
                    r = validate_ip(s,r.url,proxies={'https':proxies[ran]['proxyIp']})
                if re.search(r'公告日',r.text):
                    proxies[ran]['successful_connect_times'] =  proxies[ran]['successful_connect_times'] + 1
                    proxies[ran]['qual_ratio'] = proxies[ran]['successful_connect_times'] / proxies[ran]['connect_times']
                    break
                elif re.search(r'系統發生錯誤',r.text):
                    detail_data="系統發生錯誤"
                    breakdown = "yes"
                    break
                else:
                    if len(proxies) == 1:
                        breakdown = "yes"
                        print("proxy 不夠用了，重新獲取")
                        refresh_proxies.append(proxies.pop(ran))
                        break
                    refresh_proxies.append(proxies.pop(ran))
                    print(str(refresh_proxies[-1]['proxyIp']) + "內頁資料錯誤，還剩餘 " + str(len(proxies)) +" 個proxy")
                    pass            
            except rq.exceptions.RequestException as ex:
                if proxies[ran]['mine']==1:
                    update_mine_proxy_ip(proxies[ran],db_settings)
                #print(ex)
                error_class = ex.__class__.__name__ #取得錯誤類型
                detail = ex.args[0] #取得詳細內容
                cl, exc, tb = sys.exc_info() #取得Call Stack
                lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
                fileName = lastCallStack[0] #取得發生的檔案名稱
                lineNum = lastCallStack[1] #取得發生的行號
                funcName = lastCallStack[2] #取得發生的函數名稱
                errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
                #print(errMsg)
                if len(proxies) == 1:
                    breakdown = "yes"
                    print("proxy 不夠用了，重新獲取")
                    refresh_proxies.append(proxies.pop(ran))
                    break
                refresh_proxies.append(proxies.pop(ran))
                ran=None
                print(str(refresh_proxies[-1]['proxyIp']) + " 連接失敗，還剩餘 " + str(len(proxies)) +" 個proxy")
            except Exception as ex:
                print(ex)
                error_class = ex.__class__.__name__ #取得錯誤類型
                detail = ex.args[0] #取得詳細內容
                cl, exc, tb = sys.exc_info() #取得Call Stack
                lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
                fileName = lastCallStack[0] #取得發生的檔案名稱
                lineNum = lastCallStack[1] #取得發生的行號
                funcName = lastCallStack[2] #取得發生的函數名稱
                errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
                print("其他錯誤..")
                print(errMsg)
                ran=None
        if breakdown == "yes":
            return detail_data,detail_data_html,proxies,refresh_proxies,None

    html_doc = r.text
    soup = BeautifulSoup(html_doc, parser_detail)
    if typee=="predict":
        #政府採購預告
        detail_data_html = soup.select('div#printRange')[0]
        for i in detail_data_html.select('table'):
            for j in i.select('tr'):
                try:
                    col = j.select('th')[-1].text.strip()
                except:
                    col = j.select('td')[-2].text.strip()
                val = j.select('td')[-1].text.strip()
                detail_data[col]=val

    elif typee =="final":
        detail_data_html = soup.select('div#printArea.main table')[0]
        # 決標公告：無法決標、撤銷公告
        if soup.select('div#printArea.main div#hidden_message_id'):
            detail_data['really_final']=0
            block_data = detail_data_html.select('tr')
            if not block_data[0].text.strip():
                del block_data[0]
            if re.search(r'紅色字體表示此次更正公',block_data[0].text):
                del block_data[0]
            for j in block_data:
                try:
                    try:
                        col = j.select('th')[-1].text.strip()
                    except:
                        col = j.select('td')[-2].text.strip()
                    val = j.select('td')[-1].text.strip()
                    detail_data[col]=val
                except:
                    print("出錯了了了了了")
                    print(r.url)
                    print(j)
                    pass
                
        else:
            #決標公告：正常決標公告
            detail_data['really_final']=1
            for i in range(1,10):
                block_data = detail_data_html.select('tr.award_table_tr_'+str(i))
                if i == 1 and re.search(r'紅色字體表示此次更正公',block_data[0].text):
                    del block_data[0]
                if len(block_data) == 0 :
                    continue
                else:
                    del block_data[0]

                for j in block_data:
                    try:
                        try:
                            col = j.select('th.T11b')[-1].text.strip()
                        except:
                            col = j.select('td.newstop')[-2].text.strip()
                        val = j.select('td')[-1].text.strip()
                        detail_data[col]=val
                    except:
                        print("出錯啦啦啦啦啦")
                        print(r.url)
                        print(j)
                        continue                    
                        
    else:
        if re.search("電子競價公告",html_doc):
            keys=[]
            values=[]
            detail_data_html = soup.select('div#printArea table')[0]
            for i in detail_data_html.select('td.T11b'):
                keys.append(i.text.strip())

            for i in detail_data_html.select('td.newstop'):
                values.append(i.text.strip())

            detail_data = dict(zip(keys, values))
        else:
            #招標公告
            detail_data_html = soup.select('div#print_area table.table_block.tender_table')[0]
            for i in range(1,10):
                block_data = detail_data_html.select('tr.tender_table_tr_'+str(i))
                if len(block_data) == 0 :
                    continue
                for j in block_data:
                    try:
                        try:
                            col = j.select('th')[-1].text.strip()
                        except:
                            col = j.select('td')[-2].text.strip()
                        val = j.select('td')[-1].text.strip()
                        detail_data[col]=val
                    except:
                        pass
    return detail_data,detail_data_html,proxies,refresh_proxies,ran

In [19]:
# 抓公開徵求內頁資料
def get_searchAppeal_detail(headers,detail_connect):
    detail_data={}
    detail_data_html=""
    headers['Content-Type']='application/x-www-form-urlencoded'
    url = "https://web.pcc.gov.tw/tps/tps/tp/main/tps/tp/tp.do?method=initialAppealViewVendor&pMenu=common"
    x=0
    while True:
        try:
            x=x+1
            r = rq.post(url,data = detail_connect, headers = headers)
            if r.status_code == 200:
                x=0
                break
        except Exception as ex:
            time.sleep(1201*x)
            print(ex)
            print(f"get_searchAppeal_detail出錯，等待 {1201*x} 秒")

    html_doc = r.text
    soup = BeautifulSoup(html_doc, parser)
    if re.search(r'無符合此筆公開徵求廠商條件資料',html_doc):
        print("無符合此筆公開徵求廠商條件資料，可能公告已撤銷")
        detail_data_html = soup.select('center table')[0]
        return detail_data,detail_data_html
    
    detail_data_html = soup.select('div#printRange')[0]

    for i in detail_data_html.select('tr'):
        try:
            col = i.select('th')[-1].text.strip()
        except:
            col = i.select('td')[0].text.strip()
        val = i.select('td')[-1].text.strip()
        detail_data[col]=val
    return detail_data,detail_data_html

In [20]:
# 抓公開閱覽內頁資料
def get_publicRead_detail(detail_connect):
    x=0
    while True:
        try:
            x=x+1
            r = rq.get(detail_connect, headers = headers)
            if r.status_code == 200:
                x=0
                break
        except Exception as ex:
            time.sleep(1201*x)
            print(ex)
            print(f"get_publicRead_detail出錯，等待 {1201*x} 秒")
    
    html_doc = r.text
    soup = BeautifulSoup(html_doc, parser)
    detail_data_html_1 = soup.select('div#printRange')[0]
    detail_data_html_2 = soup.select('td#page table table table table')[2]
    detail_data={}
    for i in detail_data_html_1.select('tr'):
        col = i.select('th')[-1].text.strip()
        val = i.select('td')[-1].text.strip()
        detail_data[col]=val
    return detail_data,str(detail_data_html_1) +str(detail_data_html_2)

In [21]:
# current_date = (datetime.date.today()- datetime.timedelta(days=30)).strftime("%Y/%m/%d")
# current_date = str(int(current_date[0:4])-1911) + current_date[4:]
# print(current_date)
# s = rq.session()

# url_first = "https://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance"
# data={'method':'search','searchMethod':'true','searchTarget':'ATM','hid_1':1,'hid_2':1,'hid_3':1,'tenderStatus':'5,6,20,28,8,21,22,29,33,9,23','btnQuery':'查詢','awardAnnounceStartDate':current_date,'awardAnnounceEndDate':current_date}
# r = s.post(url_first,data = data)

# url = "https://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&execLocationArea=&pageIndex=1"
# r = s.get(url)
# soup = BeautifulSoup(r.text, parser)
# data = soup.select("div#print_area table tr")

# for i in data[1:3]:
#     #機關名稱
#     proposer_name=i.select('td')[1].text.strip()
#     print(proposer_name)
#     #標案案號
#     bid_no=re.match(r'(.*)\r\n\t', i.select('td')[2].text.strip(), flags=0).group(1)
#     print(bid_no)

In [22]:
def get_only_detail(tenderType,raw_datas,diff_seconds_data,diff_seconds_conn,headers, db_settings,No_proxy=False):
    global progress_final_detail
    global progress_tenderDeclaration_detail
    proxies = get_ip_from_db("yes",db_settings,qual_ratio=0.3)
    refresh_proxies=[]
    longg = len(raw_datas)
    loses=[]
    x=0
    ran=None
    with rq.session() as s:
        s.mount('https://', requests.adapters.HTTPAdapter(pool_connections=25, pool_maxsize=50))
        s.mount('http://', requests.adapters.HTTPAdapter(pool_connections=20, pool_maxsize=40))
        s.keep_alive = False
        for raw_data in raw_datas:
            time_start = datetime.datetime.now() #開始計時
            print("--------------------")
            try:
                while True:
                    if len(proxies)==0:
                        send_ip(refresh_proxies, db_settings,send_log=False)
                        #print(refresh_proxies)
                        proxies = get_ip_from_db("yes",db_settings,qual_ratio=0.3,mine_ip_use_interval=800)
                        # 避免既有的proxy在qual_ratio的情況下突然失效，要讓這種proxy的qual_ratio降到不會被抓取的水平要很久，所以設置如果 7次都用光proxy就重爬proxy一次
                        if len(proxies)==0 or len(loses)>7:
                            print(f"失敗總次數：{len(loses)}，強制重抓proxy！！！！！！！！！！")
                            loses=[]
                            proxies = get_proxy_ip("yes")
                            send_ip(proxies, db_settings,new = True)
                    (detail_data,detail_data_html,proxies,refresh_proxies,ran) = get_tenderDeclaration_detail(s, raw_data['detail_connect'],ran,proxies,refresh_proxies,db_settings,diff_seconds_conn,headers,typee=raw_data['tenderType'],No_proxy=No_proxy) 
                    str_detail_data_html=str(detail_data_html)
                    if len(detail_data)>0 and (re.search(raw_data['bid_no'],str_detail_data_html) or re.search(raw_data['proposer_name'],str_detail_data_html)):
                        raw_data['detail_data'] = detail_data
                        raw_data['detail_data_html'] = detail_data_html
                        print(raw_data['detail_data']['機關代碼'])
                        
                        time_end = datetime.datetime.now()    #結束計時
                        diff=time_end-time_start
                        if diff.seconds<diff_seconds_data:
                            time.sleep(diff_seconds_data-diff.total_seconds())
                        time_end = datetime.datetime.now()    #結束計時
                        time_c= time_end - time_start   #執行所花時間
                        if tenderType=='tenderDeclaration':
                            progress_tenderDeclaration_detail = f'progress_tenderDeclaration_detail 目前是第 {x+1} 項，抓取該細項內頁花費 {time_c} 秒，進度：{(x+1)*100/longg}%，總共有 {longg} 項'
                            print(progress_tenderDeclaration_detail)
                        elif tenderType=='final':
                            progress_final_detail = f'progress_final_detail 目前是第 {x+1} 項，抓取該細項內頁花費 {time_c} 秒，進度：{(x+1)*100/longg}%，總共有 {longg} 項'
                            print(progress_final_detail)
                        loses=[]
                        x=x+1
                        break
                    elif detail_data=="系統發生錯誤":
                        time_end = datetime.datetime.now()    #結束計時
                        diff=time_end-time_start
                        if diff.seconds<diff_seconds_data:
                            time.sleep(diff_seconds_data-diff.total_seconds())
                        time_end = datetime.datetime.now()    #結束計時
                        time_c= time_end - time_start   #執行所花時間
                        if tenderType=='tenderDeclaration':
                            progress_tenderDeclaration_detail = f'progress_tenderDeclaration_detail 目前是第 {x+1} 項，抓取該細項內頁花費 {time_c} 秒，系統發生錯誤，進度：{(x+1)*100/longg}%，總共有 {longg} 項'
                            print(progress_tenderDeclaration_detail)
                        elif tenderType=='final':
                            progress_final_detail = f'progress_final_detail 目前是第 {x+1} 項，抓取該細項內頁花費 {time_c} 秒，系統發生錯誤，進度：{(x+1)*100/longg}%，總共有 {longg} 項'
                            print(progress_final_detail)
                        loses=[]
                        x=x+1
                        break
                    else:
                        #print(f"失敗總次數：{len(loses)}，危險！")
                        loses.append(1)
            except Exception as ex:
                error_class = ex.__class__.__name__ #取得錯誤類型
                detail = ex.args[0] #取得詳細內容
                cl, exc, tb = sys.exc_info() #取得Call Stack
                lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
                fileName = lastCallStack[0] #取得發生的檔案名稱
                lineNum = lastCallStack[1] #取得發生的行號
                funcName = lastCallStack[2] #取得發生的函數名稱
                errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
                print(errMsg)
                print("其他錯誤..."+str(raw_data))
                pass
    send_ip(refresh_proxies, db_settings,send_log=False)
    if x>=1:
        try:
            send_db(raw_datas[0:longg],"detail_data_"+tenderType, db_settings,Notchange=False)
        except:
            print("招標決標寫入DB 失敗")
            localtime = time.localtime()
            result = time.strftime("%Y%m%d_%H%M%S", localtime)
            with open(result+'.txt', 'a+', encoding='utf-8') as file:
                file.write(str(raw_datas[0:longg]))
                file.close() 
            raise
    return x

In [23]:
#招標決標資料塞進DB
def send_db(raw_datas,data_class, db_settings,Notchange=True):
    time_start = datetime.datetime.now() #開始計時
    data_combine=[]
    # 建立Connection物件
    conn = pymysql.connect(**db_settings)
    # 建立Cursor物件
    with conn.cursor() as cursor:
      #資料表相關操作
        get_max_log_id="select MAX(UID) FROM log"
        # % 操作符只能直接用於字串(‘123’)，列表([1,2,3])、元組
        if Notchange:
            command = "INSERT INTO gov_purchase(log_UID, proposer_name, bid_no, bid_name, times, typ, clas, date, enddate, budget, ischange, detail_connect, detail_data, detail_data_html, get_data_date, tenderType)VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) on DUPLICATE KEY UPDATE log_UID = values(log_UID),get_data_date=values(get_data_date),times =values(times),typ=values(typ),clas=values(clas),date = values(date),enddate=values(enddate),budget=values(budget),ischange=values(ischange),detail_connect=values(detail_connect)"
        else:
            command = "INSERT INTO gov_purchase(log_UID, proposer_name, bid_no, bid_name, times, typ, clas, date, enddate, budget, ischange, detail_connect, detail_data, detail_data_html, get_data_date, tenderType)VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) on DUPLICATE KEY UPDATE log_UID = values(log_UID),get_data_date=values(get_data_date),times =values(times),typ=values(typ),clas=values(clas),date = values(date),enddate=values(enddate),budget=values(budget),ischange=values(ischange),detail_connect=values(detail_connect),detail_data=values(detail_data),detail_data_html=values(detail_data_html)"
        command_log = "INSERT INTO log(datetime, task, status, record_des, errmsg) VALUES(%s, %s, %s, %s, %s)"
        # 紀錄開始 (暫時取消)
#         cursor.execute(command_log, (datetime.datetime.now(), "gov_purchase", "start", data_class, ""))
        # 取得 log 的 UID
        cursor.execute(get_max_log_id)
        log_UID = str(cursor.fetchone()[0])
        # 空資料處理
        if len(raw_datas)==0:
            cursor.execute(command_log, (datetime.datetime.now(), "gov_purchase", "wrong", data_class, "nodata"))
        # 組合數據
        else:
            #抓取到的資料是以公告日期最新到最舊，這樣寫進資料庫時最新的資料會被舊的資料覆蓋住，所以加一個reversed，讓最舊的先進去
            for raw_data in reversed(raw_datas):
                data_combine.append((int(log_UID), raw_data["proposer_name"], raw_data["bid_no"], raw_data["bid_name"], raw_data["times"], raw_data["typ"], raw_data["clas"], raw_data["date"], raw_data["enddate"], raw_data["budget"], raw_data["ischange"], raw_data["detail_connect"], json.dumps(raw_data["detail_data"]), str(raw_data["detail_data_html"]),raw_data["get_data_date"], raw_data["tenderType"]))  # 注意要用两个括号扩起来

            # 執行
            try:
                cursor.executemany(command, data_combine)
            except Exception as ex:
                error_class = ex.__class__.__name__ #取得錯誤類型
                detail = ex.args[0] #取得詳細內容
                cl, exc, tb = sys.exc_info() #取得Call Stack
                lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
                fileName = lastCallStack[0] #取得發生的檔案名稱
                lineNum = lastCallStack[1] #取得發生的行號
                funcName = lastCallStack[2] #取得發生的函數名稱
                errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
                print(errMsg)
                cursor.execute(command_log, (datetime.datetime.now(), "fail", "wrong", data_class, error_class + detail + errMsg))
        # 紀錄結束
        cursor.execute(command_log, (datetime.datetime.now(), "gov_purchase", "success", data_class, str(len(raw_datas)) + " datas was completed"))


    time_end = datetime.datetime.now()    #結束計時
    time_c= time_end - time_start   #執行所花時間
    print('招標決標資料塞進DB，time cost', time_c, 's')

In [24]:
# from collections import Counter   #引入Counter
# a = ll
# b = dict(Counter(a))
# print ([key for key,value in b.items()if value > 1])  #只展示重複元素

In [25]:
# proxies = get_ip_from_db("yes",db_settings)
# global waitt
# waitt = 0
# for i in range(0,100):
#     x5 = gov_serach_crawler("tenderDeclaration",proxies,db_settings, headers = headers,start_page = 1,day_before=i)

In [26]:
def gov_serach_crawler(url_type,proxies,db_settings,headers={},cookies={},account="",password="",start_page=1,day_before=0):
    global waitt
    current_date = (datetime.date.today() - datetime.timedelta(days=day_before)).strftime("%Y/%m/%d")
    current_date = str(int(current_date[0:4])-1911) + current_date[4:]
    print("！！！！！爬取日期："+current_date)
    s = rq.session()
    s.keep_alive = True
    headers["User-Agent"] = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'
    if url_type == "tenderDeclaration":
        datarow_of_end = -1
        count_per_page = 100
        url = "https://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=basic&method=search&isSpdt=&pageIndex="
        url_first = "https://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=basic"
        data={'method':'search','searchMethod':'true','hid_1':'1','tenderType':'tenderDeclaration','tenderWay':'1,2,3,4,5,6,7,10,12','tenderDateRadio':'on','isSpdt':'N','btnQuery':'查詢','tenderEndDate':current_date,'tenderStartDate':current_date,'tenderEndDateStr':current_date,'tenderStartDateStr':current_date}
        while waitt:
                print(url_type + f"暫停休息 {60*waitt}秒 一下，有執行緒被鎖")
                time.sleep(60*waitt)
        while True:
            try:
                r = s.post(url_first, headers = headers, cookies = cookies ,data = data)
                if r.status_code<300:
                    waitt = 0
                    break
            except Exception as ex: 
                waitt = waitt + 1
                print(url_type + f"暫停休息 {1200*waitt}秒 一下，本機 IP 被鎖")
                print(ex)
                time.sleep(1200*waitt)
        global progress_tenderDeclaration
    elif url_type == "searchAppeal":  
        datarow_of_end = -1
        count_per_page = 10
        url = "https://web.pcc.gov.tw/tps/tps/tp/main/tps/tp/searchListAppealVendorCommon.do?__PageBase=0&__PageIndex="
        url_first = "https://web.pcc.gov.tw/tps/tps/tp/main/tps/tp/searchAppealVendor.do?pMenu=common"
        data={'method':'searchVendor','isVendor':'Y','startDateStr':current_date,'endDateStr':current_date}
        while waitt:
                print(url_type + f"暫停休息 {60*waitt}秒 一下，有執行緒被鎖")
                time.sleep(60*waitt)
        while True:
            try:
                r = s.post(url_first, headers = headers, cookies = cookies ,data = data)
                if r.status_code<300:
                    waitt = 0
                    break
            except Exception as ex: 
                waitt = waitt + 1
                print(url_type + f"暫停休息 {1200*waitt}秒 一下，本機 IP 被鎖")
                print(ex)
                time.sleep(1200*waitt)
        global progress_searchAppeal
    elif url_type == "publicRead":
        datarow_of_end = -2
        count_per_page = 10
        url = "https://web.pcc.gov.tw/tps/tps/tp/main/pms/tps/tp/commonPublicReadFormListPageControl.do?__PageBase=0&__PageIndex="
        url_first = "https://web.pcc.gov.tw/tps/tps/tp/main/pms/tps/tp/QueryPublicReadData.do?pMenu=common"
        data={'method':'queryPublicReadData','isVendor':'Y','startDateStr':current_date,'endDateStr':current_date}
        while waitt:
                print(url_type + f"暫停休息 {60*waitt}秒 一下，有執行緒被鎖")
                time.sleep(60*waitt)
        while True:
            try:
                r = s.post(url_first, headers = headers, cookies = cookies ,data = data)
                if r.status_code<300:
                    waitt = 0
                    break
            except Exception as ex: 
                waitt = waitt + 1
                print(url_type + f"暫停休息 {1200*waitt}秒 一下，本機 IP 被鎖")
                print(ex)
                time.sleep(1200*waitt)
        global progress_publicRead
    elif url_type == "predict":
        datarow_of_end = -1
        count_per_page = 10
        login_url = "https://web.pcc.gov.tw/pis/main/sso/login.jsp"
        login_data = {"VTI-GROUP":0,"id":account,"password":password}
        #login_data = urlencode(login_data)
        r = s.post(login_url, headers = headers, cookies = cookies ,data = login_data)
        print(r.text)
        url = ""
        url_first = "https://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=basic"
        data={'method':'searchPredict','searchMethod':'true','hid_1':1,'isSpdt':'N','btnQuery':'查詢','tenderType':'predict','tenderWay':'各式預定招標方式','tenderDateRadio':'on','tenderStartDateStr':current_date,'tenderEndDateStr':current_date,'tenderStartDate':current_date,'tenderEndDate':current_date}
        while waitt:
                print(url_type + f"暫停休息 {60*waitt}秒 一下，有執行緒被鎖")
                time.sleep(60*waitt)
        while True:
            try:
                r = s.post(url_first, headers = headers, cookies = cookies ,data = data)
                if r.status_code<300:
                    waitt = 0
                    break
            except Exception as ex: 
                waitt = waitt + 1
                print(url_type + f"暫停休息 {1200*waitt}秒 一下，本機 IP 被鎖")
                print(ex)
                time.sleep(1200*waitt)
        global progress_predict
    elif url_type == "final":  
        datarow_of_end = -1
        count_per_page = 100
        url = "https://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&execLocationArea=&pageIndex="
        url_first = "https://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance"
        data={'method':'search','searchMethod':'true','searchTarget':'ATM','hid_1':1,'hid_2':1,'hid_3':1,'tenderStatus':'5,6,20,28,8,21,22,29,33,9,23','btnQuery':'查詢','awardAnnounceStartDate':current_date,'awardAnnounceEndDate':current_date}
        while waitt:
                print(url_type + f"暫停休息 {60*waitt}秒 一下，有執行緒被鎖")
                time.sleep(60*waitt)
        while True:
            try:
                r = s.post(url_first, headers = headers, cookies = cookies ,data = data)
                if r.status_code<300:
                    waitt = 0
                    break
            except Exception as ex: 
                waitt = waitt + 1
                print(url_type + f"暫停休息 {1200*waitt}秒 一下，本機 IP 被鎖")
                print(ex)
                time.sleep(1200*waitt)
        global progress_final
    else:
        raise Exception("url_type is wrong") 


    print("外頁進入點存活")  
    refresh_proxies=[]
    raw_datas=[]
    for ii in range(start_page,2000):
        #s = rq.session()
        while waitt:
                print(url_type + f"暫停休息 {60*waitt}秒 一下，有執行緒被鎖_phase2")
                time.sleep(60*waitt)
        while True:
            try:
                r = s.get(url + str(ii), headers = headers)
                print("外頁列表頁存活")  
                if r.status_code == 200:
                    waitt = 0
                    break
            except Exception as ex: 
                print(url_type + f"暫停休息 {1200*waitt}秒 一下，本機 IP 被鎖_phase2")
                print(ex)
                waitt = waitt + 1
                time.sleep(1200*waitt)
        print("頁面前往："+str(r.url))
        html_doc = r.text
        if re.search(r'無符合條件資料',html_doc):
            print("無符合條件資料")
            break
        soup = BeautifulSoup(html_doc, parser)
        
        time_start = datetime.datetime.now()
        if url_type == "tenderDeclaration":
            data = soup.select('div#print_area table tr')
            all_data = int(data[datarow_of_end].select('span')[-1].text)
            xxx = 0
            for i in data[1:datarow_of_end]:
                t_s = datetime.datetime.now()
                xxx = xxx + 1
                progress_tenderDeclaration = "這是第 " + str(ii) + " 頁，的第 "+ str(xxx) + " 項，總共有 " + str(all_data) + " 項，目前進度是 " + str(((ii-1)*count_per_page+xxx)/all_data*100) + " %。"
                print(progress_tenderDeclaration)
                # 取得資料日期
                get_data_date = datetime.datetime.now().date()
                #項次
                no=i.select('td')[1].text.strip()
                #機關名稱
                proposer_name=i.select('td')[1].text.strip()
                #標案案號
                bid_no=re.match(r'(.*)\r\n\t', i.select('td')[2].text.strip(), flags=0).group(1)
                # 是否更正
                if i.select('td')[2].select('font'):
                    ischange=1
                else:
                    ischange=0  
                # 標案名稱    
                bid_name=i.select('td')[2].select('a')[0].text.strip()
                # 傳輸次數
                times=i.select('td')[3].text.strip()
                # 招標方式
                typ=i.select('td')[4].text.strip()
                # 採購性質
                clas=i.select('td')[5].text.strip()
                # 公告日期
                date=i.select('td')[6].text.strip()
                # 截止投標
                enddate=i.select('td')[7].text.strip()
                # 預算金額
                budget=i.select('td')[8].text.strip()
                # 內文連結
                detail_connect = "https://web.pcc.gov.tw/tps/" + i.select('td')[2].select('a')[0]['href'][3:]
                detail_data ={}
                detail_data_html=""
#                 while True:
#                     if len(proxies)==0:
#                         send_ip(refresh_proxies, db_settings,send_log=False)
#                         refresh_proxies=[]
#                         proxies = get_ip_from_db("yes",db_settings,qual_ratio=0.3)
#                         if proxies == []:
#                             xx = get_proxy_ip("yes")
#                             send_ip(xx, db_settings)
#                             proxies = get_ip_from_db("yes",db_settings)
#                         #(detail_data,detail_data_html,proxies,refresh_proxies) = get_tenderDeclaration_detail(detail_connect,proxies,refresh_proxies,headers) 
#                     (detail_data,detail_data_html,proxies,refresh_proxies) = get_tenderDeclaration_detail(detail_connect,proxies,refresh_proxies,headers) 
#                     str_detail_data_html=str(detail_data_html)
#                     if len(detail_data)>0 and (re.search(bid_no,str_detail_data_html) or re.search(proposer_name,str_detail_data_html)):
#                         print(detail_data['機關代碼'])
#                         break
                t_e = datetime.datetime.now()
                time_se= t_e - t_s
                print('此細項已抓取完成，總耗時：', time_se, 's')
                print("------------------------------------")
                raw_datas.append({'no': no,'proposer_name': proposer_name,'bid_no': bid_no,'bid_name': bid_name,'times': times,'typ': typ,'clas': clas,'date': date,'enddate': enddate,'budget': budget,'ischange': ischange,'detail_connect':detail_connect,'detail_data':detail_data,'detail_data_html':detail_data_html,'get_data_date':get_data_date,'tenderType':"tenderDeclaration"})
        elif url_type == "searchAppeal":
            data = soup.select('center table:nth-child(1) tr')
            all_data = int(data[datarow_of_end].select('span')[-1].text)
            xxxx = 0
            for i in data[17:datarow_of_end]:
                xxxx = xxxx + 1
                progress_searchAppeal = "這是第 " + str(ii) +" 頁，的第 "+ str(xxxx) + " 項，總共有 " + str(all_data) + " 項，目前進度是 " + str(((ii-1)*count_per_page+xxxx)/all_data*100) + " %。"
                print(progress_searchAppeal)
                # 取得資料日期
                get_data_date = datetime.datetime.now().date()
                #項次
                no=i.select('td')[0].text.strip()
                #機關名稱
                proposer_name=i.select('td')[1].text.strip()
                #標案案號
                bid_no=i.select('td')[2].text.strip()
                # 是否更正
                ischange=0  
                # 標案名稱    
                bid_name=i.select('td')[3].text.strip()
                # 傳輸次數 (公告次數)
                times=i.select('td')[4].text.strip()
                # 招標方式
                typ=""
                # 採購性質
                clas=""
                # 公告日期
                date=re.match(r'(.*)\s*－\s*(.*)', i.select('td')[5].text.strip(), flags=0).group(1).strip()
                # 截止投標
                enddate=re.match(r'(.*)\s*－\s*(.*)', i.select('td')[5].text.strip(), flags=0).group(2).strip()
                # 預算金額
                budget=""
                # 內文連結
                detail_connect = {}
                detail_connects = i.select('td.T12 input')
                for ij in detail_connects:
                    try:
                        detail_connect[ij['id']]=ij['value']
                    except:
                        break
                detail_connect=urlencode(detail_connect)
                (detail_data,detail_data_html) = get_searchAppeal_detail(headers,detail_connect)
                raw_datas.append({'no': no,'proposer_name': proposer_name,'bid_no': bid_no,'bid_name': bid_name,'times': times,'typ': typ,'clas': clas,'date': date,'enddate': enddate,'budget': budget,'ischange': ischange,'detail_data':detail_data, 'detail_connect':detail_connect,'detail_data_html':detail_data_html,'get_data_date':get_data_date,'tenderType':"searchAppeal"})
        elif url_type == "publicRead":
            data = soup.select('td#page table:nth-child(3) tr')
            all_data = int(data[datarow_of_end].select('span')[-1].text)
            xxxxx=0
            for i in data[15:datarow_of_end]:
                xxxxx = xxxxx + 1
                progress_publicRead = "這是第 " + str(ii) +" 頁，的第 "+ str(xxxxx) + " 項，總共有 " + str(all_data) + " 項，目前進度是 " +str(((ii-1)*count_per_page+xxxxx)/all_data*100) + " %。"
                print(progress_publicRead)
                # 取得資料日期
                get_data_date = datetime.datetime.now().date()
                # 項次
                no=i.select('td')[0].text.strip()
                # 機關名稱
                proposer_name=i.select('td')[1].text.strip()
                # 標案案號
                bid_no=i.select('td')[2].text.strip()
                # 內文連結
                detail_connect = "https://web.pcc.gov.tw" + i.select('td')[2].select('a')[0]['href']
                # 是否更正
                ischange=0  
                # 標案名稱    
                bid_name=i.select('td')[3].text.strip()
                # 傳輸次數 (公告次數)
                times=i.select('td')[4].text.strip()
                # 招標方式
                typ=""
                # 採購性質
                clas=""
                # 公告日期
                date=re.match(r'(.*)\s*─\s*(.*)', i.select('td')[5].text.strip(), flags=0).group(1).strip()
                # 截止投標
                enddate=re.match(r'(.*)\s*─\s*(.*)', i.select('td')[5].text.strip(), flags=0).group(2).strip()
                # 預算金額
                budget=""
                (detail_data,detail_data_html)=get_publicRead_detail(detail_connect)
                detail_data=""
                raw_datas.append({'no': no,'proposer_name': proposer_name,'bid_no': bid_no,'bid_name': bid_name,'times': times,'typ': typ,'clas': clas,'date': date,'enddate': enddate,'budget': budget,'ischange': ischange,'detail_connect':detail_connect,'detail_data':detail_data,'detail_data_html':detail_data_html,'get_data_date':get_data_date,'tenderType':"publicRead"})
        elif url_type == "predict" : 
            data = soup.select("div#print_area table tr")
            all_data = int(data[datarow_of_end].select('span')[-1].text)
            xxxxx = 0
            for i in data[1:datarow_of_end]:
                xxxxx = xxxxx + 1
                progress_predict = "這是第 " + str(ii) +" 頁，的第 "+ str(xxxxx) + " 項，總共有 " + str(all_data) + " 項，目前進度是 " + str(((ii-1)*count_per_page+xxxxx)/all_data*100) + " %。"
                print(progress_predict)
                # 取得資料日期
                get_data_date = datetime.datetime.now().date()
                #項次
                no=i.select('td')[0].text.strip()
                #機關名稱
                proposer_name=i.select('td')[3].text.strip()
                #標案案號
                bid_no=i.select('td')[1].text.strip()
                # 是否更正
                ischange=0  
                # 標案名稱    
                bid_name=i.select('td')[2].select('a')[0].text.strip()
                # 傳輸次數
                times=""
                # 招標方式
                typ=i.select('td')[4].text.strip()
                # 採購性質
                clas=""
                # 公告日期
                date=i.select('td')[5].text.strip()
                # 截止投標
                enddate=""
                # 預算金額
                budget=""
                # 內文連結
                detail_connect = "https://web.pcc.gov.tw/tps/" + i.select('td')[6].select('a')[0]['href'][5:]
                print(detail_connect)
                while True:
                    if len(proxies)==0:
                        send_ip(refresh_proxies, db_settings,send_log=False)
                        refresh_proxies=[]
                        proxies = get_ip_from_db("yes",db_settings,qual_ratio=0.3)
                        if proxies == []:
                            xx = get_proxy_ip("yes")
                            send_ip(xx, db_settings)
                            proxies = get_ip_from_db("yes",db_settings)
                        #(detail_data,detail_data_html,proxies,refresh_proxies) = get_tenderDeclaration_detail(detail_connect,proxies,refresh_proxies,headers) 
                    (detail_data,detail_data_html,proxies,refresh_proxies) = get_tenderDeclaration_detail(detail_connect,proxies,refresh_proxies,headers,typee="predict") 
                    str_detail_data_html=str(detail_data_html)
                    if len(proxies)>0 and (re.search(bid_no,str_detail_data_html) or re.search(proposer_name,str_detail_data_html)):
                        print("predict detail 通過")
                        if re.search(r'勞務類',str_detail_data_html):
                            clas=clas+'勞務類'
                        if re.search(r'工程類',str_detail_data_html):
                            clas=clas+'工程類'
                        if re.search(r'財物類',str_detail_data_html):
                            clas=clas+'財物類'
                raw_datas.append({'no': no,'proposer_name': proposer_name,'bid_no': bid_no,'bid_name': bid_name,'times': times,'typ': typ,'clas': clas,'date': date,'enddate': enddate,'budget': budget,'ischange': ischange,'detail_connect':detail_connect,'detail_data':detail_data,'detail_data_html':detail_data_html,'get_data_date':get_data_date,'tenderType':"predict"})
        elif url_type == "final" : 
            data = soup.select("div#print_area table tr")
            all_data = int(data[datarow_of_end].select('span')[-1].text)
            xxxxxx = 0
            for i in data[1:datarow_of_end]:
                t_s = datetime.datetime.now()
                xxxxxx = xxxxxx + 1
                progress_final = "這是第 " + str(ii) +" 頁，的第 "+ str(xxxxxx) + " 項，總共有 " + str(all_data) + " 項，目前進度是 " + str(((ii-1)*count_per_page+xxxxxx)/all_data*100) + " %。"
                print(progress_final)
                # 取得資料日期
                get_data_date = datetime.datetime.now().date()
                #項次
                no=i.select('td')[0].text.strip()
                #機關名稱
                proposer_name=i.select('td')[1].text.strip()
                #標案案號
                bid_no=re.match(r'(.*)\r\n\t', i.select('td')[2].text.strip(), flags=0).group(1)
                # 是否更正
                if i.select('td')[2].select('font'):
                    ischange=1
                else:
                    ischange=0   
                # 標案名稱    
                bid_name=i.select('td')[2].select('u')[0].text.strip()
                # 傳輸次數
                times=""
                # 這樣可以從內頁資料抓傳輸次數： 上面有定義 dicMemberCheck 可以吐出字典裡對應key的value
#                 if detail_data['ischange'] ==0 and not re.search(r'限制性招標',detail_data['招標方式']) and not re.search(r'選擇性招標',detail_data['招標方式']):
#                     print(detail_data['新增公告傳輸次數'])
                
                # 招標方式
                typ=i.select('td')[3].text.strip()
                # 採購性質
                clas=i.select('td')[4].text.strip()
                # 公告日期
                date=i.select('td')[5].text.strip()
                # 截止投標
                enddate=""
                # 預算金額
                budget=i.select('td')[6].text.strip()
                # 內文連結
                detail_connect = "https://web.pcc.gov.tw/tps/" + i.select('td')[2].select('a')[0]['href'][3:]
                detail_data={}
                detail_data_html=""
#                 while True:
#                     if len(proxies)==0:
#                         send_ip(refresh_proxies, db_settings,send_log=False)
#                         refresh_proxies=[]
#                         proxies = get_ip_from_db("yes",db_settings,qual_ratio=0.3)
#                         if proxies == []:
#                             xx = get_proxy_ip("yes")
#                             send_ip(xx, db_settings)
#                             proxies = get_ip_from_db("yes",db_settings)
#                         #(detail_data,detail_data_html,proxies,refresh_proxies) = get_tenderDeclaration_detail(detail_connect,proxies,refresh_proxies,headers) 
#                     (detail_data,detail_data_html,proxies,refresh_proxies) = get_tenderDeclaration_detail(s, detail_connect,proxies,refresh_proxies,headers,typee="final") 
#                     str_detail_data_html=str(detail_data_html)
#                     if len(detail_data)>0 and (re.search(bid_no,str_detail_data_html) or re.search(proposer_name,str_detail_data_html)):
#                         print(detail_data['機關代碼'])
#                         break
                t_e = datetime.datetime.now()
                time_se= t_e - t_s
                print('此細項已抓取完成，總耗時：', time_se, 's')
                print("------------------------------------")
                raw_datas.append({'no': no,'proposer_name': proposer_name,'bid_no': bid_no,'bid_name': bid_name,'times': times,'typ': typ,'clas': clas,'date': date,'enddate': enddate,'budget': budget,'ischange': ischange,'detail_connect':detail_connect,'detail_data':detail_data,'detail_data_html':detail_data_html,'get_data_date':get_data_date,'tenderType':"final"})
        time_end = datetime.datetime.now()
        time_c= time_end - time_start
        print('此分頁抓取總耗時：', time_c, 's')
        #all_data = int(data[datarow_of_end].select('span')[-1].text)
        print(all_data)
        print(ii*count_per_page)
        if ii*count_per_page >= all_data:
            if url_type == "tenderDeclaration":
                del progress_tenderDeclaration
            elif url_type == "searchAppeal":
                del progress_searchAppeal
            elif url_type == "publicRead":    
                del progress_publicRead
            elif url_type == "predict":    
                del progress_predict
            elif url_type == "final":    
                del progress_final
            break
    return raw_datas

In [27]:
def search_by_date(s,start_date, end_date,if_publish,headers):
    r = s.get("https://web.pcc.gov.tw/prkms/prms-viewDailyTenderListClient.do?root=tps",headers=headers)
    html_doc = r.text
    if r.status_code == 500 or re.search(r'The server encountered an internal error or misconfiguration and was unable to complete your request',html_doc):
        raise Exception("打不進去header頁") 
        return ""
    soup = BeautifulSoup(html_doc, parser)
    publishs_href=[]
    publishs_date=[]
    not_publishs_href=[]
    not_publishs_date=[]
    left = soup.select('td#page table tr td:nth-child(2) li a')
    right = soup.select('td#page table tr td:nth-child(4) li a')
    for i in left:
        publishs_href.append(i['href'])
        y=str(int(re.match(r'(.*)年(.*)月(.*)日',i['title']).group(1))+1911)
        m=re.match(r'(.*)年(.*)月(.*)日',i['title']).group(2)
        d=re.match(r'(.*)年(.*)月(.*)日',i['title']).group(3)
        ymd = datetime.datetime.strptime("-".join([y,m,d]), "%Y-%m-%d")
        publishs_date.append(ymd)
    for i in right:
        not_publishs_href.append(i['href'])
        y=str(int(re.match(r'(.*)年(.*)月(.*)日',i['title']).group(1))+1911)
        m=re.match(r'(.*)年(.*)月(.*)日',i['title']).group(2)
        d=re.match(r'(.*)年(.*)月(.*)日',i['title']).group(3)
        ymd = datetime.datetime.strptime("-".join([y,m,d]), "%Y-%m-%d")
        not_publishs_date.append(ymd)
    for i in publishs_date:
        publishs_start_date_index = publishs_date.index(i)
        if i < start_date:
            break
    for i in not_publishs_date:
        not_publishs_start_date_index = not_publishs_date.index(i)
        if i < start_date:
            break
    for i in publishs_date:
        publishs_end_date_index = publishs_date.index(i)
        if i <= end_date:
            break
    for i in not_publishs_date:
        not_publishs_end_date_index = not_publishs_date.index(i)
        if i <= end_date:
            break

    if not_publishs_start_date_index<not_publishs_end_date_index:
        raise Exception("開始日期不可大於結束日期")
    if publishs_start_date_index<publishs_end_date_index:
        raise Exception("開始日期不可大於結束日期")
    if start_date > not_publishs_date[0] or start_date > publishs_date[0]:
        raise Exception("無資料可爬，初始日期大於網站最新日期")
    
    total_hrefs=[]
    if if_publish=="all" or if_publish=="yes":
        for i in range(0,len(publishs_href[publishs_end_date_index:publishs_start_date_index])):
            total_href={'href':publishs_href[publishs_end_date_index+i],'date':publishs_date[publishs_end_date_index+i],'ispublish':1}
            total_hrefs.append(total_href)
    if if_publish=="all" or if_publish=="no":
        for i in range(0,len(not_publishs_href[not_publishs_end_date_index:not_publishs_start_date_index])):
            total_href={'href':not_publishs_href[not_publishs_end_date_index+i],'date':not_publishs_date[not_publishs_end_date_index+i],'ispublish':0}
            total_hrefs.append(total_href)
    return total_hrefs

In [28]:
def date_serach_crawler(s,search_date,headers):
    url_head = "https://web.pcc.gov.tw/prkms/"
    len_search_date=len(search_date)
    global progress_date_serach
    if len_search_date==0:
        return
    xxxxxx = 0
    for i in search_date:
        xxxxxx = xxxxxx + 1
        progress_date_serach = "這是第 " + str(xxxxxx) + " 個，總共有 " + str(len_search_date) + " 個要爬"
        print(search_date.index(i))
        r = s.get(url_head + i['href'], headers=headers)
        html_doc = r.text
        soup = BeautifulSoup(html_doc, parser)
        i['detail_data'] = soup.select('td#page table table table table tr td')[0]
    del progress_date_serach
    return search_date

In [29]:
def send_db_2(raw_datas,data_class, db_settings):
    time_start = datetime.datetime.now() #開始計時
    data_combine=[]
    try:
        # 建立Connection物件
        conn = pymysql.connect(**db_settings)
        # 建立Cursor物件
        with conn.cursor() as cursor:
          #資料表相關操作
            get_max_log_id="select MAX(UID) FROM log"
            # % 操作符只能直接用於字串(‘123’)，列表([1,2,3])、元組
            command = "INSERT INTO date_search_by_date(log_UID, date, ispublish, upper, lower, href)VALUES(%s, %s, %s, %s, %s, %s) on DUPLICATE KEY UPDATE log_UID = values(log_UID),date=values(date),ispublish =values(ispublish),upper=values(upper),lower=values(lower),href = values(href)"
            command_log = "INSERT INTO log(datetime, task, status, record_des, errmsg) VALUES(%s, %s, %s, %s, %s)"
            # 紀錄開始 (暫時取消)
#             cursor.execute(command_log, (datetime.datetime.now(), "date_search_by_date", "start", data_class, ""))
            # 取得 log 的 UID
            cursor.execute(get_max_log_id)
            log_UID = str(cursor.fetchone()[0])
            # 組合數據
            for raw_data in raw_datas:
                data_combine.append((int(log_UID), raw_data["date"], raw_data["ispublish"], "", raw_data["detail_data"], raw_data["href"]))  # 注意要用两个括号扩起来

            # 執行
            try:
                cursor.executemany(command, data_combine)
            except Exception as ex:
                print(ex)
                error_class = ex.__class__.__name__ #取得錯誤類型
                detail = ex.args[0] #取得詳細內容
                cl, exc, tb = sys.exc_info() #取得Call Stack
                lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
                fileName = lastCallStack[0] #取得發生的檔案名稱
                lineNum = lastCallStack[1] #取得發生的行號
                funcName = lastCallStack[2] #取得發生的函數名稱
                errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
                print(errMsg)
                cursor.execute(command_log, (datetime.datetime.now(), "date_search_by_date", "fail", data_class, error_class + detail + errMsg))
            # 紀錄結束
            cursor.execute(command_log, (datetime.datetime.now(), "date_search_by_date", "success", data_class, str(len(raw_datas)) + " datas was completed"))

    except Exception as ex:
        print(ex)

    time_end = datetime.datetime.now()    #結束計時
    time_c= time_end - time_start   #執行所花時間
    print('time cost', time_c, 's')

In [30]:
# proxies = get_ip_from_db("yes",db_settings)
# # x4 = gov_serach_crawler("predict",proxies,db_settings, headers = headers)
# # send_db(x4,"predict", db_settings)
# x5 = gov_serach_crawler("final",proxies,db_settings, headers = headers)
# send_db(x5,"final", db_settings)

In [ ]:
#雙重 thread
my_host=input("請輸入主機ip ( 若要從本機啟動，可輸入127.0.0.1 )：")
#account=input("請輸入政府採購網帳號，目前請隨便輸入我還沒做：")
#password=input("請輸入政府採購網密碼，目前請隨便輸入我還沒做：")
my_host=str(my_host)
app = Flask(__name__)
# db_settings = {
#     "host": "127.0.0.1",
#     "port": 3306,
#     "user": "root",
#     "password": "As123459362",
#     "db": "pttcrawler",
#     "charset": "utf8",
#     "autocommit":True
# }
db_settings = {
    "host": "60.250.109.71",
    "port": 23306,
    "user": "xuan",
    "password": "Qaz123",
    "db": "tender",
    "charset": "utf8mb4",
    "autocommit":True
}
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
    "Accept-Encoding": "gzip, deflate, br", 
    "Accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6", 
    "Host": "web.pcc.gov.tw",  #目標網站 
    "Sec-Fetch-Dest": "document", 
    "Sec-Fetch-Mode": "navigate", 
    "Sec-Fetch-Site": "same-origin", 
    "Upgrade-Insecure-Requests": "?1", 
     #使用者代理
}
command_log = "INSERT INTO log(datetime, task, status, record_des, errmsg) VALUES(%s, %s, %s, %s, %s)"

@app.route('/', methods=['GET'])
def test():
#     ip = request.remote_addr
#     print(ip)
#     conn = pymysql.connect(**db_settings)
#     # 建立Cursor物件
#     with conn.cursor() as cursor:
#         if_finish_command="SELECT status FROM log where task ='api_process_list' order by UID DESC"
#         cursor.execute(if_finish_command)
#         if cursor.fetchone() is None:
#             cursor.execute(command_log, (datetime.datetime.now(), "api_process_list", "initial", "", ""))
#         cursor.execute(if_finish_command)
#         if_finish = str(cursor.fetchone()[0])
#     if re.search(r'finish', if_finish, flags=0) or if_finish == "initial":
#         wording ='<p>恭喜!網站建置成功!</p><p>api說明：</p><p>1.&nbsp;/thread：查看當前執行緒狀態，無須參數 (get請求)</p><p>2.&nbsp;/start_task_all：開始執行爬蟲，若要強制重啟，參數帶 restart = 1 (get請求)</p><p>3.&nbsp;/date_serach_crawler：公告日期執行爬蟲，參數帶 start_date、end_date、if_publish (get請求)<br />start_date、end_date =&gt; 以字串型態帶日期，ex:20210125，if_publish =&gt; 以字串形式帶 all | yes | no </p><p>http://127.0.0.1:5000/date_serach_crawler?start_date=20210128&end_date=20210204&if_publish=yes</p><p>4.&nbsp;/progress：查看當前執行緒進度，無須參數 (get請求)</p><p>5. /proxy : 從免費代理網站抓取proxy進資料庫，無須參數 (get請求)'
#         status_code = 200
#     else:
#         wording ="I am busy now, please wait"
#         status_code = 500
        
    wording ='<p>恭喜!網站建置成功!</p><p>api說明：</p><p>1.&nbsp;/thread：查看當前執行緒狀態，無須參數 (get請求)</p><p>2.&nbsp;/start_task_list：執行爬蟲：抓列表資料，若要強制重啟，參數帶 restart = 1；若要設定爬取的初始頁面，參數帶 start_page=[x,x,x,x] ([招標公告頁數、公開徵求頁數、公開閱覽頁數、決標公告頁數])；若想要設定爬取 N 天前的資料，參數要帶 day_before，以 int 的方式放值，ex: day_before=0  (get請求)</p><p>http://127.0.0.1:5000/start_task_list?restart=1&start_page=[1,1,1,1]&day_before=0</p><p><p>3.&nbsp;/start_task_detail：執行爬蟲：抓內頁資料，要晚於 /start_task_list 5分鐘，可設定連接等待時間，參數帶 (float)diff_seconds_conn (要求每一次嘗試連接內頁的時間需大於多少秒)、(float)diff_seconds_data (要求確實取得內頁資料的時間需大於多少秒) (get請求)</p><p>http://127.0.0.1:5000/start_task_detail?diff_seconds_conn=0&diff_seconds_data=0&No_proxy=0&spilt=(2,1)</p>4.&nbsp;/date_serach_crawler：執行爬蟲：依公告日期查詢，參數帶 start_date、end_date、if_publish (get請求)<br />start_date、end_date =&gt; 以字串型態帶日期，ex:20210125，if_publish =&gt; 以字串形式帶 all | yes | no </p><p>http://127.0.0.1:5000/date_serach_crawler?start_date=20210128&end_date=20210204&if_publish=yes</p><p>5.&nbsp;/progress：查看當前執行緒進度，無須參數 (get請求)</p><p>6. /proxy : 從免費代理網站抓取proxy進資料庫，無須參數 (get請求)' 
    status_code = 200

    return wording,status_code

@app.route('/thread', methods=['GET'])
def thread_status():
    wording =""
    wording2 =""
    wording3 =""
    wording4 =""
    wording5=""
    wording6=""
    code =200
    print(str(threading.enumerate()))
    try:
        if str(threading.enumerate()).find("thread_tenderDeclaration_searchAppeal") >=0:
            wording = "當前正在執行 thread_tenderDeclaration_searchAppeal 執行緒"
            code = 500
    except Exception as ex:
        print(ex)
        pass
    
    try:
        if str(threading.enumerate()).find("thread_date_serach_crawler") >=0:
            wording2 = "當前正在執行 thread_date_serach_crawler 執行緒"
            code = 500
    except Exception as ex:
        print(ex)
        pass
    
    try:
        if str(threading.enumerate()).find("thread_proxy_crawler") >=0:
            wording3 = "當前正在執行 thread_proxy_crawler 執行緒"
            code = 500
    except Exception as ex:
        print(ex)
        pass
    
    try:
        if str(threading.enumerate()).find("thread_final_datesearch_publicRead") >=0:
            wording4 = "當前正在執行 thread_final_datesearch_publicRead 執行緒"
            code = 500
    except Exception as ex:
        print(ex)
        pass
    
    try:
        if str(threading.enumerate()).find("thread_tenderDeclaration_detail") >=0:
            wording5 = "當前正在執行 thread_tenderDeclaration_detail 執行緒"
            code = 500
    except Exception as ex:
        print(ex)
        pass
    
    try:
        if str(threading.enumerate()).find("thread_final_detail") >=0:
            wording6 = "當前正在執行 thread_final_detail 執行緒"
            code = 500
    except Exception as ex:
        print(ex)
        pass
        
    return wording + " " + wording2 + " " + wording3 + " " + wording4 + " " + wording5 + " " + wording6,code
        
@app.route('/proxy', methods=['GET'])
def proxy():
    def proxyy():
        p_yes=get_proxy_ip("yes")
        #p_no=get_proxy_ip("no")
        send_ip(p_yes, db_settings ,"only https",new = True)
        #send_ip(p_no, db_settings ,"only http",new = True)
    try:
        if str(threading.enumerate()).find("thread_proxy_crawler") >=0:
            return "已經在爬 proxy 了，別吵",500
    except Exception as ex:
        print(ex)
        pass    
    
    thread_proxy_crawler = threading.Thread(target=proxyy, kwargs={},name="thread_proxy_crawler")
    thread_proxy_crawler.start()
    return "started",200
    
@app.route('/start_task_list', methods=['GET'])
def start_task_all():
    if request.args:
        dd= request.args.to_dict()
        if set(['start_page','restart','day_before']) >= set(dd.keys()):
            print("傳參成功")
        else:
            return "傳參失敗，參數只允許"+' start_page '+'restart '+'day_before',500
    def do_work(start_page,day_before,db_settings,headers):
        conn = pymysql.connect(**db_settings)
        with conn.cursor() as cursor:
            cursor.execute(command_log, (datetime.datetime.now(), "api_process_list", str(ip) + " start", "tenderDeclaration_searchAppeal", f"tenderDeclaration 從第 {start_page[0]} 頁開始爬，searchAppeal 從第 {start_page[1]} 頁開始爬"))
        time_start = time.time()
        proxies = get_ip_from_db("yes",db_settings)
        print("=====1=======")
        
        try:
            x1 = gov_serach_crawler("tenderDeclaration",proxies,db_settings, headers = headers,start_page = int(start_page[0]),day_before=day_before)
            send_db(x1,"tenderDeclaration", db_settings)
        except Exception as ex:
            print(ex)
            error_class = ex.__class__.__name__ #取得錯誤類型
            detail = ex.args[0] #取得詳細內容
            cl, exc, tb = sys.exc_info() #取得Call Stack
            lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
            fileName = lastCallStack[0] #取得發生的檔案名稱
            lineNum = lastCallStack[1] #取得發生的行號
            funcName = lastCallStack[2] #取得發生的函數名稱
            errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
            print(errMsg)
            conn = pymysql.connect(**db_settings)
            with conn.cursor() as cursor:
                cursor.execute(command_log, (datetime.datetime.now(), "api_process_list", "wrong", "tenderDeclaration", errMsg))
        print("=====2=======")
        try:
            x2 = gov_serach_crawler("searchAppeal",proxies,db_settings, headers = headers,start_page = int(start_page[1]),day_before=day_before)
            send_db(x2,"searchAppeal", db_settings)
        except Exception as ex:
            print(ex)
            error_class = ex.__class__.__name__ #取得錯誤類型
            detail = ex.args[0] #取得詳細內容
            cl, exc, tb = sys.exc_info() #取得Call Stack
            lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
            fileName = lastCallStack[0] #取得發生的檔案名稱
            lineNum = lastCallStack[1] #取得發生的行號
            funcName = lastCallStack[2] #取得發生的函數名稱
            errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
            print(errMsg)
            conn = pymysql.connect(**db_settings)
            with conn.cursor() as cursor:
                cursor.execute(command_log, (datetime.datetime.now(), "api_process_list", "wrong", "searchAppeal", errMsg))
        
#         try:
#             x4 = gov_serach_crawler("predict",proxies,db_settings, headers = headers, account=account,password=password)
#             send_db(x4,"predict", db_settings)
#         except Exception as ex:
#             print(ex)
#             error_class = ex.__class__.__name__ #取得錯誤類型
#             detail = ex.args[0] #取得詳細內容
#             cl, exc, tb = sys.exc_info() #取得Call Stack
#             lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
#             fileName = lastCallStack[0] #取得發生的檔案名稱
#             lineNum = lastCallStack[1] #取得發生的行號
#             funcName = lastCallStack[2] #取得發生的函數名稱
#             errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
#             print(errMsg)
#             conn = pymysql.connect(**db_settings)
#             with conn.cursor() as cursor:
#                 cursor.execute(command_log, (datetime.datetime.now(), "api_process_list", "wrong", "predict", errMsg))
        print("=====4=======")
        time_d = time.time()
        time_c = time_d - time_start
        print("！！！！！  thread_tenderDeclaration_searchAppeal 總執行時間 "+str(time_c) +" 秒")
        conn = pymysql.connect(**db_settings)
        with conn.cursor() as cursor:
            cursor.execute(command_log, (datetime.datetime.now(), "api_process_list", str(ip) + " finish", "tenderDeclaration_searchAppeal", ""))
            
    def do_work_final(start_page,day_before,db_settings,headers):
        conn = pymysql.connect(**db_settings)
        with conn.cursor() as cursor:
            cursor.execute(command_log, (datetime.datetime.now(), "api_process_list", str(ip) + " start", "final_publicRead_datesearch", f"final 從第 {start_page[3]} 頁開始爬，publicRead 從第 {start_page[2]} 頁開始爬，datesearch爬當日"))
        time_start = time.time()
        proxies = get_ip_from_db("yes",db_settings)
        try:
            x5 = gov_serach_crawler("final",proxies,db_settings, headers = headers,start_page = int(start_page[3]),day_before=day_before)
            send_db(x5,"final", db_settings)
        except Exception as ex:
            print(ex)
            error_class = ex.__class__.__name__ #取得錯誤類型
            detail = ex.args[0] #取得詳細內容
            cl, exc, tb = sys.exc_info() #取得Call Stack
            lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
            fileName = lastCallStack[0] #取得發生的檔案名稱
            lineNum = lastCallStack[1] #取得發生的行號
            funcName = lastCallStack[2] #取得發生的函數名稱
            errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
            print(errMsg)
            conn = pymysql.connect(**db_settings)
            with conn.cursor() as cursor:
                cursor.execute(command_log, (datetime.datetime.now(), "api_process_list", "wrong", "final", errMsg))
                
        try:
            x3 = gov_serach_crawler("publicRead",proxies,db_settings, headers = headers,start_page = int(start_page[3]),day_before=day_before)
            send_db(x3,"publicRead", db_settings)
        except Exception as ex:
            print(ex)
            error_class = ex.__class__.__name__ #取得錯誤類型
            detail = ex.args[0] #取得詳細內容
            cl, exc, tb = sys.exc_info() #取得Call Stack
            lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
            fileName = lastCallStack[0] #取得發生的檔案名稱
            lineNum = lastCallStack[1] #取得發生的行號
            funcName = lastCallStack[2] #取得發生的函數名稱
            errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
            print(errMsg)
            conn = pymysql.connect(**db_settings)
            with conn.cursor() as cursor:
                cursor.execute(command_log, (datetime.datetime.now(), "api_process_list", "wrong", "publicRead", errMsg))

        try:
            s = rq.session()
            xx = search_by_date(s,datetime.datetime.today()- datetime.timedelta(days=1), datetime.datetime.today(),"all", headers = headers)
            print(xx)
            xxx = date_serach_crawler(s,xx,headers)
            send_db_2(xxx,"date_search_by_date", db_settings)
        except Exception as ex:
            print(ex)
            error_class = ex.__class__.__name__ #取得錯誤類型
            detail = ex.args[0] #取得詳細內容
            cl, exc, tb = sys.exc_info() #取得Call Stack
            lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
            fileName = lastCallStack[0] #取得發生的檔案名稱
            lineNum = lastCallStack[1] #取得發生的行號
            funcName = lastCallStack[2] #取得發生的函數名稱
            errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
            print(errMsg)
            conn = pymysql.connect(**db_settings)
            with conn.cursor() as cursor:
                cursor.execute(command_log, (datetime.datetime.now(), "api_process_date_serach", "wrong", "date_serach_crawler", error_class + detail + errMsg))
      
        time_d = time.time()
        time_c = time_d - time_start
        print("！！！！！  thread_final_datesearch_publicRead 總執行時間 "+str(time_c) +" 秒")
        conn = pymysql.connect(**db_settings)
        with conn.cursor() as cursor:
            cursor.execute(command_log, (datetime.datetime.now(), "api_process_list", str(ip) + " finish", "final_publicRead_datesearch", ""))
    
    #驗證用圖片資料夾
    path = './temp_validate_img'
    if os.path.exists(path):
        shutil.rmtree (path)
        os.mkdir(path)
    else:
        os.mkdir(path)
    #資料庫log檢查
    ip = request.remote_addr
    conn = pymysql.connect(**db_settings)
    if_finish_command="SELECT status FROM log where task ='api_process_list' order by UID DESC"
    with conn.cursor() as cursor:
        cursor.execute(if_finish_command)
        if cursor.fetchone() is None:
            cursor.execute(command_log, (datetime.datetime.now(), "api_process_list", "initial", "", ""))
        cursor.execute(if_finish_command)
        if_finish = str(cursor.fetchone()[0])
    
    # 不判斷資料庫log強制重啟
    if re.search(r'finish', if_finish, flags=0) or re.search(r'wrong', if_finish, flags=0) or if_finish == "initial":
        pass
    elif "restart" in request.args:
        try:
            if request.args.get("restart")==1:
                pass
        except:
            return "若想要重啟爬蟲，參數要帶restart，值請放 1 "
    else:
        return "資料庫未收到結束log，可帶參數 restart=1 忽略資料庫log判斷",500
    #執行序之間任一ip被鎖block全部
    global waitt
    waitt = 0
    
    day_before=0
    if 'day_before' in request.args:
        try:
            day_before = eval(request.args.get("day_before"))
        except:
            return "若想要設定爬取 N 天前的資料，參數要帶 day_before，以 int 的方式放值，ex: day_before=0 "

    
    #設定起始頁面
    start_page=[1,1,1,1]
    if 'start_page' in request.args:
        try:
            start_page = eval(request.args.get("start_page"))
            if len(start_page) !=4:
                return "start_page 參數長度不足 4 (須設定 4 個：tenderDeclaration、searchAppeal、publicRead、final)，無須爬取的項目可輸入1999！"
            else:
                print("頁數參數接收成功")
        except:
            return "若想要設定爬取的初始頁數，參數要帶 start_page，以 list 的方式放值=>[招標公告頁數,公開徵求頁數,公開閱覽頁數,決標公告頁數]，ex: start_page=[1,1,1,1] "
    print(f"tenderDeclaration 從第 {start_page[0]} 頁開始爬取")
    print(f"searchAppeal 從第 {start_page[1]} 頁開始爬取")
    print(f"publicRead 從第 {start_page[2]} 頁開始爬取")
    print(f"final 從第 {start_page[3]} 頁開始爬取")
    
    try:
        if str(threading.enumerate()).find("thread_tenderDeclaration_searchAppeal") >= 0 :
            wording0="thread_tenderDeclaration_searchAppeal 執行緒正在執行；"
        else:
            thread_tenderDeclaration_searchAppeal = threading.Thread(target=do_work, kwargs={'start_page':start_page,'day_before':day_before,'db_settings': db_settings,'headers': headers},name="thread_tenderDeclaration_searchAppeal")
            thread_tenderDeclaration_searchAppeal.start()
            wording0="啟動 thread_tenderDeclaration_searchAppeal 執行緒；"
    except Exception as ex:
        print(ex)
        pass
    
    try:
        if str(threading.enumerate()).find("thread_final_datesearch_publicRead") >= 0 :
            wording1="thread_final_datesearch_publicRead 執行緒正在執行"
        else:
            thread_final_datesearch_publicRead = threading.Thread(target=do_work_final, kwargs={'start_page':start_page,'day_before':day_before,'db_settings': db_settings,'headers': headers},name="thread_final_datesearch_publicRead")
            thread_final_datesearch_publicRead.start()
            wording1="啟動 thread_final_datesearch_publicRead 執行緒"
    except Exception as ex:
        print(ex)
        pass

    return wording0+wording1,200

@app.route('/start_task_detail', methods=['GET'])
def start_task_detail():
    path = './temp_validate_img'
    if os.path.exists(path):
        shutil.rmtree (path)
        os.mkdir(path)
    else:
        os.mkdir(path)
    ip = request.remote_addr

    def do_work_detail(tenderType,spilt,diff_seconds_data,diff_seconds_conn,db_settings,headers,No_proxy):
        global progress_final_detail
        progress_final_detail =''
        global progress_tenderDeclaration_detail
        progress_tenderDeclaration_detail =''
        
        conn = pymysql.connect(**db_settings)
        get_data="SELECT * FROM tender.gov_purchase where (detail_data_html ='' and detail_data like '%{}%') and tenderType='"+tenderType+"'"
        # 建立Cursor物件
        with conn.cursor(pymysql.cursors.DictCursor) as cursor:
          #資料表相關操作
            cursor.execute(get_data)
            raw_datas_all = cursor.fetchall()
            if spilt:
                print(spilt)
                if len(raw_datas_all)>=spilt[0]:
                    first_part = int(len(raw_datas_all)/spilt[0]*(spilt[1]-1))
                    second_part = int(len(raw_datas_all)/spilt[0]*spilt[1])
                    print(f'{tenderType} 共抓出 {len(raw_datas_all)} 個未處理，切分 {spilt[0]} 段，此為第 {spilt[1]} 段，取第 {first_part} 個到 {second_part} 個，共 {second_part-first_part} 個，進行爬取')
                    raw_datas_all = raw_datas_all[first_part:second_part]
            cursor.execute(command_log, (datetime.datetime.now(), "api_process_detail", str(ip) + " start", tenderType, f'{tenderType} 切分 {spilt[0]} 段，此為第 {spilt[1]} 段，取第 {first_part} 個到 {second_part} 個，共 {len(raw_datas_all)} 個，進行爬取'))
            raw_datas=raw_datas_all[0:1000]
        
        try:
            #限制嘗試抓取次數，避免死機
            x=0
            while len(raw_datas)>0:
                
                ll = get_only_detail(tenderType,raw_datas,diff_seconds_data,diff_seconds_conn,headers, db_settings,No_proxy=No_proxy)
                # get_only_detail會回傳成功抓取的筆數 (ll)
                if ll>=1:
                    x = 0
                else:
                    x = x + 1 
                    
                #避免過長抓取失敗
                del raw_datas_all[0:1000]
                if len(raw_datas_all)>=1000:
                    raw_datas=raw_datas_all[0:1000]
                else:
                    raw_datas=raw_datas_all
                    
#                 conn = pymysql.connect(**db_settings)
#                 # 建立Cursor物件
#                 with conn.cursor(pymysql.cursors.DictCursor) as cursor:
#                   #資料表相關操作
#                     cursor.execute(get_data)
#                     raw_datas = cursor.fetchall()
#                 if len(raw_datas) ==0:
#                     print('都抓完了！！！！')
#                     break
#                 else:
                if x>=2:
                    print(f'{tenderType} 出現 {len(raw_datas)} 筆無法抓取內頁資料')
                    break
            conn = pymysql.connect(**db_settings)
            with conn.cursor() as cursor:
                cursor.execute(command_log, (datetime.datetime.now(), "api_process_detail", str(ip)+"finish", tenderType, ""))

        except Exception as ex:
            print(ex)
            error_class = ex.__class__.__name__ #取得錯誤類型
            detail = ex.args[0] #取得詳細內容
            cl, exc, tb = sys.exc_info() #取得Call Stack
            lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
            fileName = lastCallStack[0] #取得發生的檔案名稱
            lineNum = lastCallStack[1] #取得發生的行號
            funcName = lastCallStack[2] #取得發生的函數名稱
            errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
            print(errMsg)
            conn = pymysql.connect(**db_settings)
            with conn.cursor() as cursor:
                cursor.execute(command_log, (datetime.datetime.now(), "api_process_detail", "wrong", "tenderDeclaration_final_detail", errMsg))
        
        if tenderType=='tenderDeclaration':
            del progress_tenderDeclaration_detail
        elif tenderType=='final':
            del progress_final_detail
            
    if request.args:
        dd = request.args.to_dict()
        print(dd)
        if set(['diff_seconds_data','diff_seconds_conn','No_proxy','spilt']) >= set(dd.keys()):
            print("傳參成功")
        else:
            return "傳參失敗，參數只允許"+' diff_seconds_data '+' diff_seconds_conn ' + 'spilt',500
        try:
            #限制取得內頁資料的時間
            diff_seconds_data = dicMemberCheck('diff_seconds_data',dd)
            if diff_seconds_data !="":
                diff_seconds_data=float(diff_seconds_data)
            else:
                diff_seconds_data=0
            print(f"diff_seconds_data {diff_seconds_data}")    
            #限制每一次嘗試取得內頁資料的時間
            diff_seconds_conn = dicMemberCheck('diff_seconds_conn',dd)
            if diff_seconds_conn !="":
                diff_seconds_conn=float(diff_seconds_conn) 
            else:
                diff_seconds_conn=0
            print(f"diff_seconds_conn {diff_seconds_conn}")   
            #設定用本機ip爬取內頁資料
            No_proxy = dicMemberCheck('No_proxy',dd)
            if No_proxy =='1':
                No_proxy = True
            else:
                No_proxy = False
            print(f"No_proxy {No_proxy}")
            #切分段數
            spilt = dicMemberCheck('spilt',dd)
            if spilt !="" :
                spilt=eval(spilt)
                if len(spilt)==2 and spilt[0]>=spilt[1]:
            else:
                spilt = False
            print(f"spilt {spilt}")
        except Exception as ex:
            print(ex)
            return "參數轉換錯誤",500
    
    
    if str(threading.enumerate()).find("thread_tenderDeclaration_detail") >= 0 :
        wording1="thread_tenderDeclaration_detail 執行緒正在執行"
    else:
        thread_tenderDeclaration_detail = threading.Thread(target=do_work_detail, kwargs={'tenderType':'tenderDeclaration','diff_seconds_data':diff_seconds_data,'diff_seconds_conn':diff_seconds_conn,'headers':headers,'db_settings': db_settings,'No_proxy':No_proxy,'spilt':spilt},name="thread_tenderDeclaration_detail")
        thread_tenderDeclaration_detail.start()
        wording1="啟動 thread_tenderDeclaration_detail 執行緒"
        
    if str(threading.enumerate()).find("thread_final_detail") >= 0 :
        wording2="thread_final_detail 執行緒正在執行"
    else:
        thread_final_detail = threading.Thread(target=do_work_detail, kwargs={'tenderType':'final','diff_seconds_data':diff_seconds_data,'diff_seconds_conn':diff_seconds_conn,'headers':headers,'db_settings': db_settings,'No_proxy':No_proxy,'spilt':spilt},name="thread_final_detail")
        thread_final_detail.start()
        wording2="啟動 thread_final_detail 執行緒"
        
    return wording1 +" "+ wording2,200



@app.route('/progress', methods=['GET'])
def get_progress():
    wording={}
    try:
        wording['tenderDeclaration'] = progress_tenderDeclaration
    except:
        wording['tenderDeclaration'] = "tenderDeclaration 任務未啟動"
        
    try:
        wording['searchAppeal'] = progress_searchAppeal
    except:
        wording['searchAppeal'] = "searchAppeal 任務未啟動"
        
    try:
        wording['publicRead'] = progress_publicRead
    except:
        wording['publicRead'] = "publicRead 任務未啟動"
        
    try:
        wording['predict'] = progress_predict
    except:
        wording['predict'] = "predict 任務未啟動"
    
    try:
        wording['final'] = progress_final
    except:
        wording['final'] = "final 任務未啟動"
        
    try:
        wording['date_serach'] = progress_date_serach
    except:
        wording['date_serach'] = "date_serach 任務未啟動"

    try:
        wording['tenderDeclaration_detail'] = progress_tenderDeclaration_detail
    except:
        wording['tenderDeclaration_detail'] = "tenderDeclaration_detail 任務未啟動"
        
    try:
        wording['final_detail'] = progress_final_detail
    except:
        wording['final'] = "final_detail 任務未啟動"
        
    return wording,200

# @app.route('/test', methods=['GET'])
# def tesddt():
#     if request.args:
#         dd= request.args.to_dict()
#         if set(['start_date','end_date','']) >= set(dd.keys()):
#             return "傳參成功",200
#         else:
#             return "傳參失敗",200


@app.route('/date_serach_crawler', methods=['GET'])
def date_serach():
    if request.args:
        dd= request.args.to_dict()
        if set(['start_date','end_date','if_publish']) >= set(dd.keys()):
            print("傳參成功")
        else:
            return "傳參失敗，參數只允許"+' start_date '+'end_date '+'if_publish',500
    headers["User-Agent"] = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'
    def do_work_2(start_date,end_date,if_publish,db_settings,headers):
        s = rq.session()
        try:
            xx = search_by_date(s,start_date, end_date,if_publish,headers)
            print(xx)
            xxx = date_serach_crawler(s,xx,headers)
            send_db_2(xxx,"date_search_by_date", db_settings)
        except Exception as ex:
            print(ex)
            error_class = ex.__class__.__name__ #取得錯誤類型
            detail = ex.args[0] #取得詳細內容
            cl, exc, tb = sys.exc_info() #取得Call Stack
            lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
            fileName = lastCallStack[0] #取得發生的檔案名稱
            lineNum = lastCallStack[1] #取得發生的行號
            funcName = lastCallStack[2] #取得發生的函數名稱
            errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
            print(errMsg)
            conn = pymysql.connect(**db_settings)
            with conn.cursor() as cursor:
                cursor.execute(command_log, (datetime.datetime.now(), "api_process_date_serach", "wrong", "date_serach_crawler", error_class + detail + errMsg))
        conn = pymysql.connect(**db_settings)
        with conn.cursor() as cursor:
            cursor.execute(command_log, (datetime.datetime.now(), "api_process_date_serach", str(ip) + " finish", "date_serach_crawler", ""))
        
        
    try:
        if str(threading.enumerate()).find("thread_start_date_serach_crawler") >=0:
            return "當前執行緒正在執行",500
    except Exception as ex:
        print(ex)
        pass
    
    try:
        start_date = str(request.args.get("start_date"))
        start_date = datetime.datetime.strptime(start_date, "%Y%m%d")
    except Exception as ex:    
        return "start_date 參數格式錯誤",500
    
    try:
        end_date = str(request.args.get("end_date"))
        end_date = datetime.datetime.strptime(end_date, "%Y%m%d")
    except Exception as ex:    
        return "end_date 參數格式錯誤",500
    
    try:
        if_publish = str(request.args.get("if_publish"))
        if not (if_publish == "yes" or if_publish == "no" or if_publish == "all"):
            return "if_publish 參數格式錯誤",500
    except Exception as ex:
        return "if_publish 參數沒帶",500
        
    
    ip = request.remote_addr
    conn = pymysql.connect(**db_settings)
    with conn.cursor() as cursor:
        if_finish_command="SELECT status FROM log where task ='api_process_date_serach' order by UID DESC"
        cursor.execute(if_finish_command)
        if cursor.fetchone() is None:
            cursor.execute(command_log, (datetime.datetime.now(), "api_process_date_serach", "initial", "", ""))
        cursor.execute(if_finish_command)
        if_finish = str(cursor.fetchone()[0])
    
    print(if_finish)    
    if re.search(r'finish', if_finish, flags=0) or re.search(r'wrong', if_finish, flags=0) or if_finish == "initial":
        pass
    else:
        wording ="資料庫未收到結束log"
        status_code = 500
        return wording,status_code
    
    conn = pymysql.connect(**db_settings)
    with conn.cursor() as cursor:
        cursor.execute(command_log, (datetime.datetime.now(), "api_process_date_serach", str(ip) + " start", "date_serach_crawler", ""))
    
    thread_date_serach_crawler = threading.Thread(target=do_work_2, kwargs={'start_date': start_date,'end_date': end_date,'if_publish':if_publish,'db_settings':db_settings,'headers':headers},name="thread_date_serach_crawler")
    thread_date_serach_crawler.start()    
    return "started",200


if __name__ == '__main__':
    app.debug = False
    app.config['JSON_AS_ASCII'] = False
    app.run(host=my_host, port=5000)
    #serve(app, host=my_host, port=5000)

請輸入主機ip ( 若要從本機啟動，可輸入127.0.0.1 )：
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Apr/2022 00:36:59] "GET /start_task_detail?diff_seconds_conn=6&diff_seconds_data=6&No_proxy=1&spilt=(2,2) HTTP/1.1" 200 -


{'diff_seconds_conn': '6', 'diff_seconds_data': '6', 'No_proxy': '1', 'spilt': '(2,2)'}
傳參成功
diff_seconds_data 6.0
diff_seconds_conn 6.0
No_proxy True
spilt (2,2)
spilt (2, 2)


127.0.0.1 - - [05/Apr/2022 00:38:25] "GET /progress HTTP/1.1" 200 -


(2, 2)
tenderDeclaration 共抓出 9239 個未處理，切分 2 段，此為第 2 段，取第 4619 個到 9239 個，共 4620 個，進行爬取
(2, 2)
final 共抓出 14244 個未處理，切分 2 段，此為第 2 段，取第 7122 個到 14244 個，共 7122 個，進行爬取
1232
True
1232
True
--------------------
--------------------
本機 IP 準備執行驗證
本機 IP 準備執行驗證
通過驗證
3.9.20.1通過驗證

progress_final_detail 目前是第 1 項，抓取該細項內頁花費 0:00:07.034734 秒，進度：0.1%，總共有 1000 項
--------------------
3.79.4.93
progress_tenderDeclaration_detail 目前是第 1 項，抓取該細項內頁花費 0:00:07.136285 秒，進度：0.1%，總共有 1000 項
--------------------
本機 IP 準備執行驗證
本機 IP 準備執行驗證
通過驗證
3.9.20.1
通過驗證progress_final_detail 目前是第 2 項，抓取該細項內頁花費 0:00:06.824122 秒，進度：0.2%，總共有 1000 項

--------------------
3.79.4.93
progress_tenderDeclaration_detail 目前是第 2 項，抓取該細項內頁花費 0:00:06.951722 秒，進度：0.2%，總共有 1000 項
--------------------
本機 IP 準備執行驗證
3.9.20.1
progress_final_detail 目前是第 3 項，抓取該細項內頁花費 0:00:06.280284 秒，進度：0.3%，總共有 1000 項
--------------------
通過驗證
3.79.4.93.1
progress_tenderDeclaration_detail 目前是第 3 項，抓取該細項內頁花費 0:00:07.139237 秒，進度：0.3%，總共有 1000 項
--------------------
3.9

...

本機 IP 準備執行驗證
本機 IP 準備執行驗證
通過驗證
3.79.8.78
progress_final_detail 目前是第 695 項，抓取該細項內頁花費 0:00:07.286655 秒，進度：69.5%，總共有 1000 項
--------------------
通過驗證
3.13.31.45
progress_tenderDeclaration_detail 目前是第 697 項，抓取該細項內頁花費 0:00:07.230779 秒，進度：69.7%，總共有 1000 項
--------------------
3.79.8.73
progress_final_detail 目前是第 696 項，抓取該細項內頁花費 0:00:06.197412 秒，進度：69.6%，總共有 1000 項
--------------------
本機 IP 準備執行驗證
通過驗證
3.13.31.45
progress_tenderDeclaration_detail 目前是第 698 項，抓取該細項內頁花費 0:00:06.789290 秒，進度：69.8%，總共有 1000 項
--------------------
3.79.5.13
progress_final_detail 目前是第 697 項，抓取該細項內頁花費 0:00:06.233266 秒，進度：69.7%，總共有 1000 項
--------------------
本機 IP 準備執行驗證
通過驗證
3.13.31.48
progress_tenderDeclaration_detail 目前是第 699 項，抓取該細項內頁花費 0:00:06.781005 秒，進度：69.9%，總共有 1000 項
--------------------
本機 IP 準備執行驗證
通過驗證
3.79.4.6
progress_final_detail 目前是第 698 項，抓取該細項內頁花費 0:00:07.225596 秒，進度：69.8%，總共有 1000 項
--------------------
本機 IP 準備執行驗證
通過驗證
3.13.31.48
progress_tenderDeclaration_detail 目前是第 700 項，抓取該細項內頁花費 0:00:06.975

tenderDeclaration暫停休息 60秒 一下，有執行緒被鎖，目前時間：20220405_092519
tenderDeclaration暫停休息 60秒 一下，有執行緒被鎖，目前時間：20220405_092619
tenderDeclaration暫停休息 60秒 一下，有執行緒被鎖，目前時間：20220405_092719
tenderDeclaration暫停休息 60秒 一下，有執行緒被鎖，目前時間：20220405_092819
tenderDeclaration暫停休息 60秒 一下，有執行緒被鎖，目前時間：20220405_092919
tenderDeclaration暫停休息 60秒 一下，有執行緒被鎖，目前時間：20220405_093019
tenderDeclaration暫停休息 60秒 一下，有執行緒被鎖，目前時間：20220405_093120
tenderDeclaration暫停休息 60秒 一下，有執行緒被鎖，目前時間：20220405_093220
tenderDeclaration暫停休息 60秒 一下，有執行緒被鎖，目前時間：20220405_093320
tenderDeclaration暫停休息 60秒 一下，有執行緒被鎖，目前時間：20220405_093420
tenderDeclaration暫停休息 60秒 一下，有執行緒被鎖，目前時間：20220405_093520
tenderDeclaration暫停休息 60秒 一下，有執行緒被鎖，目前時間：20220405_093620
tenderDeclaration暫停休息 60秒 一下，有執行緒被鎖，目前時間：20220405_093720
tenderDeclaration暫停休息 60秒 一下，有執行緒被鎖，目前時間：20220405_093820
本機 ip 遭到封鎖
final暫停休息 1600秒 一下，本機 IP 被鎖，目前時間：20220405_093834
('Connection aborted.', OSError(0, 'Error'))
tenderDeclaration暫停休息 120秒 一下，有執行緒被鎖，目前時間：20220405_093920
tenderDeclaration暫停休息 120秒 一下，有執行緒被鎖，目前

本機 IP 準備執行驗證
通過驗證
3.76.60.3.6
progress_tenderDeclaration_detail 目前是第 739 項，抓取該細項內頁花費 0:00:06.929235 秒，進度：73.9%，總共有 1000 項
--------------------
3.95.25
progress_final_detail 目前是第 756 項，抓取該細項內頁花費 0:00:06.217801 秒，進度：75.6%，總共有 1000 項
--------------------
本機 IP 準備執行驗證
通過驗證
3.76.60.18
progress_tenderDeclaration_detail 目前是第 740 項，抓取該細項內頁花費 0:00:06.857203 秒，進度：74.0%，總共有 1000 項
--------------------
3.95.25
progress_final_detail 目前是第 757 項，抓取該細項內頁花費 0:00:06.233430 秒，進度：75.7%，總共有 1000 項
--------------------
本機 IP 準備執行驗證
本機 IP 準備執行驗證
通過驗證
3.76.60.96.5
progress_tenderDeclaration_detail 目前是第 741 項，抓取該細項內頁花費 0:00:07.226134 秒，進度：74.1%，總共有 1000 項
--------------------
通過驗證
3.95.32
progress_final_detail 目前是第 758 項，抓取該細項內頁花費 0:00:07.269504 秒，進度：75.8%，總共有 1000 項
--------------------
本機 IP 準備執行驗證
本機 IP 準備執行驗證
通過驗證
3.76.60.96.2
progress_tenderDeclaration_detail 目前是第 742 項，抓取該細項內頁花費 0:00:06.902628 秒，進度：74.2%，總共有 1000 項
--------------------
通過驗證
3.95.32
progress_final_detail 目前是第 759 項，抓取該細項內頁花費 0:00:07.20226

本機 IP 準備執行驗證
通過驗證
3.9.28
progress_tenderDeclaration_detail 目前是第 768 項，抓取該細項內頁花費 0:00:07.462516 秒，進度：76.8%，總共有 1000 項
--------------------
3.76.54.63
progress_final_detail 目前是第 786 項，抓取該細項內頁花費 0:00:06.226191 秒，進度：78.6%，總共有 1000 項
--------------------
本機 IP 準備執行驗證
通過驗證
3.9.31
progress_tenderDeclaration_detail 目前是第 769 項，抓取該細項內頁花費 0:00:06.803928 秒，進度：76.9%，總共有 1000 項
--------------------
3.76.54.64
progress_final_detail 目前是第 787 項，抓取該細項內頁花費 0:00:06.411878 秒，進度：78.7%，總共有 1000 項
--------------------
本機 IP 準備執行驗證
通過驗證
3.92.1.18
progress_tenderDeclaration_detail 目前是第 770 項，抓取該細項內頁花費 0:00:07.036834 秒，進度：77.0%，總共有 1000 項
--------------------
3.76.54.64
progress_final_detail 目前是第 788 項，抓取該細項內頁花費 0:00:06.258642 秒，進度：78.8%，總共有 1000 項
--------------------
本機 IP 準備執行驗證
通過驗證
A.25.3
progress_tenderDeclaration_detail 目前是第 771 項，抓取該細項內頁花費 0:00:07.039420 秒，進度：77.1%，總共有 1000 項
--------------------
3.76.54.64
progress_final_detail 目前是第 789 項，抓取該細項內頁花費 0:00:06.273084 秒，進度：78.9%，總共有 1000 項
------------------

本機 IP 準備執行驗證
通過驗證
3.92.5.5
progress_tenderDeclaration_detail 目前是第 798 項，抓取該細項內頁花費 0:00:07.002073 秒，進度：79.8%，總共有 1000 項
--------------------
A.11.10
progress_final_detail 目前是第 818 項，抓取該細項內頁花費 0:00:06.243016 秒，進度：81.8%，總共有 1000 項
--------------------
本機 IP 準備執行驗證
通過驗證
3.92.5.5
progress_tenderDeclaration_detail 目前是第 799 項，抓取該細項內頁花費 0:00:07.074026 秒，進度：79.9%，總共有 1000 項
--------------------
本機 IP 準備執行驗證
通過驗證
A.11.10
progress_final_detail 目前是第 819 項，抓取該細項內頁花費 0:00:07.394307 秒，進度：81.9%，總共有 1000 項
--------------------
本機 IP 準備執行驗證
通過驗證
3.92.1.54
progress_tenderDeclaration_detail 目前是第 800 項，抓取該細項內頁花費 0:00:07.100269 秒，進度：80.0%，總共有 1000 項
--------------------
本機 IP 準備執行驗證
通過驗證
3.76.55.55
progress_final_detail 目前是第 820 項，抓取該細項內頁花費 0:00:06.919077 秒，進度：82.0%，總共有 1000 項
--------------------
本機 IP 準備執行驗證
通過驗證
3.92.3.14
progress_tenderDeclaration_detail 目前是第 801 項，抓取該細項內頁花費 0:00:06.997697 秒，進度：80.1%，總共有 1000 項
--------------------
本機 IP 準備執行驗證
通過驗證
3.76.55
progress_final_detail 目前是第 821 項，抓取該細項內頁花費 0:0

In [32]:
# thread_list=[]
# thread_list_index=[0]
# thread = 4
# for i in range(1,thread):
#     thread_list_index.append(int(len(raw_datas)/thread*i))
# thread_list_index.append(len(raw_datas))
# print(thread_list_index)
# ind=0
# while ind<=thread:
#     ind=ind+1
#     thread_list.append(threading.Thread(target=do_work_detail, kwargs={'raw_datas':raw_datas[thread_list_index[ind]:thread_list_index[ind+1]],'diff_seconds_data':diff_seconds_data,'diff_seconds_conn':diff_seconds_conn,'headers':headers,'db_settings': db_settings},name=f"thread_tenderDeclaration_final_detail_{thread_list_index[ind]}"))

In [33]:
# import requests as rq
# import ssl
# s=rq.session()
# URKK = "https://web.pcc.gov.tw/pis/main/pis/client/index.do"
# URKK = "https://www.youtube.com/watch?v=HMPVdcU9ZKg"
# ssl._create_default_https_context = ssl._create_unverified_context
# headers = rq.utils.default_headers()
# r = s.get(URKK,verify=False,timeout=(10,20))
# r.close()
# r.request.headers

In [34]:
db_settings = {
    "host": "60.250.109.71",
    "port": 23306,
    "user": "xuan",
    "password": "Qaz123",
    "db": "tender",
    "charset": "utf8mb4",
    "autocommit":True
}

headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
    "Accept-Encoding": "gzip, deflate, br", 
    "Accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6", 
    "Host": "web.pcc.gov.tw",  #目標網站 
    "Sec-Fetch-Dest": "document", 
    "Sec-Fetch-Mode": "navigate", 
    "Sec-Fetch-Site": "same-origin", 
    "Upgrade-Insecure-Requests": "?1", 
     #使用者代理
}
# global waitt
# waitt =0
# proxies = get_ip_from_db("yes",db_settings)
# x5 = gov_serach_crawler("final",proxies,db_settings, headers = headers)
# send_db(x5,"final", db_settings)

In [35]:
# db_settings = {
#     "host": "60.250.109.71",
#     "port": 23306,
#     "user": "xuan",
#     "password": "Qaz123",
#     "db": "tender",
#     "charset": "utf8mb4",
#     "autocommit":True
# }

# headers = {
#     "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
#     "Accept-Encoding": "gzip, deflate, br", 
#     "Accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6", 
#     "Host": "web.pcc.gov.tw",  #目標網站 
#     "Sec-Fetch-Dest": "document", 
#     "Sec-Fetch-Mode": "navigate", 
#     "Sec-Fetch-Site": "same-origin", 
#     "Upgrade-Insecure-Requests": "?1", 
#      #使用者代理
# }

# time_start = time.time()
# proxies = get_ip_from_db("yes",db_settings)
# try:
#     x1 = gov_serach_crawler("tenderDeclaration",proxies,db_settings, headers = headers)
#     x2 = gov_serach_crawler("searchAppeal",proxies,db_settings, headers = headers)
#     x3 = gov_serach_crawler("publicRead",proxies,db_settings, headers = headers)
#     time_d = time.time()
# except Exception as ex:
#     print(ex)
#     time_d = time.time()
# time_c = time_d - time_start
# print(time_c)
# send_db(x1, db_settings)
# send_db(x2, db_settings)
# send_db(x3, db_settings)

In [36]:
# s = rq.session()
# ss = datetime.datetime(2020, 1, 28)
# e = datetime.datetime(2021, 6, 28)
# xx = search_by_date(s,ss, e,'no')
# xxx = date_serach_crawler(s,xx)

# db_settings = {
#     "host": "60.250.109.71",
#     "port": 23306,
#     "user": "xuan",
#     "password": "Qaz123",
#     "db": "tender",
#     "charset": "utf8mb4",
#     "autocommit":True
# }
# data_class="date_search_by_date"
# send_db_2(xxx,data_class, db_settings)